# 도로명주소 이상치 확인하기

## import

In [109]:
# visualization
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
fe = fm.FontEntry(fname=r'font/NanumGothic.otf', name='NanumBarunGothic')
fm.fontManager.ttflist.insert(0, fe)
plt.rcParams.update({'font.size': 10, 'font.family': 'NanumBarunGothic'})
plt.rc('font', family='NanumBarunGothic')


# utils
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
import warnings;warnings.filterwarnings('ignore')


from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Model
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.linear_model import LinearRegression
import lightgbm as lgb


from sklearn import metrics
from sklearn.metrics import mean_squared_error as MSE

import eli5
from eli5.sklearn import PermutationImportance

# 컬럼 화면 설정
pd.set_option('display.max_columns', 150)
pd.set_option('display.max_rows', 50)

## 데이터 로드

In [95]:
# 데이터 로드
train = pd.read_csv('../data/match_build_seoul.csv')
df = pd.read_csv('../data/trainset_0715.csv')
jm_df = pd.read_csv('../data/jibun_xy_by_naver.csv')
jm_train = pd.read_csv('../data/train_add_newXY.csv')
jm_test = pd.read_csv('../data/test_add_newXY.csv')
test = pd.read_csv('../data/test.csv')

### 데이터 확인

In [96]:
# shape 확인
print(train.shape)
print(df.shape)
print(jm_df.shape)
print(jm_train.shape)
print(jm_test.shape)
print(test.shape)

(599943, 34)
(1128094, 116)
(8953, 4)
(1118822, 54)
(9272, 53)
(9272, 51)


In [97]:
# 내용확인
df.head(1)

,시군구,번지,본번,부번,아파트명,전용면적(㎡),계약년월,계약일,층,건축년도,도로명,target,is_test,번지주소,id,계약년월일,구,동,도로명주소,계약년,번호,k-아파트코드,k-아파트명,"k-단지분류(아파트,주상복합등등)",kapt도로명주소,주소(시군구),주소(읍면동),주소(도로명),k-전체동수,k-전체세대수,...,생산자물가지수,소비자물가지수,서울특별시_가계대출,서울특별시_주택담보대출,서울특별시_주택매매거래량,서울특별시_아파트매매거래량,전국매매가격지수,전국전세가격지수,서울특별시_주택인허가실적,서울특별시_주택착공실적,서울특별시_주택준공실적,서울특별시_주택미분양,미분양_서울특별시,지번주소,공시가격기반단지명,동수_y,세대수,사용승인일자,소형학원수,중형학원수,대형학원수,종합학원수,입시학원수,외국어학원수,예체능학원수,독서실수,정보학원수,특수교육학원수,직업기술학원수,기타학원수
0,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.97,201712,8,3,1987,언주로 3,124000.0,0,서울특별시 강남구 개포동 658-1,서울특별시 강남구 개포동 658-1 3층,2017-12-08,강남구,개포동,서울특별시 강남구 언주로 3,2017,40291.0,A10023565,개포6차우성아파트,아파트,서울특별시 강남구 언주로 3,강남구,개포동,언주로,8.0,270.0,...,99.09,97.698,282438.7,176991.0,13740.0,8484.0,87.701,93.042,113131.0,13157.0,5968.0,45.0,45.0,서울특별시 강남구 개포동 658-1,개포6차우성아파트1동~8동,8.0,270.0,19871121.0,39.0,3.0,0.0,2.0,23.0,1.0,14.0,0.0,0.0,0.0,1.0,1.0


In [99]:
jm_train.head(1)

,시군구,번지,본번,부번,아파트명,전용면적(㎡),계약년월,계약일,층,건축년도,도로명,해제사유발생일,등기신청일자,거래유형,중개사소재지,"k-단지분류(아파트,주상복합등등)",k-전화번호,k-팩스번호,단지소개기존clob,k-세대타입(분양형태),k-관리방식,k-복도유형,k-난방방식,k-전체동수,k-전체세대수,k-건설사(시공사),k-시행사,k-사용검사일-사용승인일,k-연면적,k-주거전용면적,k-관리비부과면적,k-전용면적별세대현황(60㎡이하),k-전용면적별세대현황(60㎡~85㎡이하),k-85㎡~135㎡이하,k-135㎡초과,k-홈페이지,k-등록일자,k-수정일자,고용보험관리번호,경비비관리형태,세대전기계약방법,청소비관리형태,건축면적,주차대수,기타/의무/임대/임의=1/2/3/4,단지승인일,사용허가여부,관리비 업로드,좌표X,좌표Y,단지신청일,target,좌표X_2,좌표Y_2
0,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.97,201712,8,3,1987,언주로 3,NaN,,-,-,아파트,025776611,025776673,NaN,분양,자치관리,계단식,개별난방,8.0,270.0,우성건설,모름,1987-11-21 00:00:00.0,22637.0,20204.0,22637.0,20.0,250.0,0.0,NaN,NaN,2022-11-09 20:10:43.0,2023-09-23 17:21:41.0,NaN,직영,단일계약,직영,4858.0,262.0,임의,2022-11-17 13:00:29.0,Y,N,127.05721,37.476763,2022-11-17 10:19:06.0,124000,127.056859,37.476276


In [100]:
jm_df.head(2) # 사용 안함

,Unnamed: 0,지번주소,좌표X,좌표Y
0,0,서울특별시 강남구 개포동 1164-12,127.052949,37.473193
1,1,서울특별시 강남구 개포동 1164-13,127.053094,37.473287


In [101]:
# 결측치확인
jm_train[['번지','좌표X_2', '좌표Y_2']].isna().sum()

번지       0
좌표X_2    0
좌표Y_2    0
dtype: int64

### 지번 및 좌표데이터 추가를 위해 jm_train, jm_test 병합

In [102]:
jm = pd.concat([jm_train, jm_test], axis=0)

In [103]:
jm.head(1) # 데이터 확인
plus_df = jm[['번지','좌표X_2', '좌표Y_2']] # 필요한 column들만 추출

In [104]:
plus_df = plus_df.reset_index(drop=True) # 인덱스 초기화

In [106]:
# df에 '열이름' + '_jm'로 열 추가
df[['번지_jm','좌표X_jm','좌표Y_jm']] = plus_df[['번지','좌표X_2','좌표Y_2']]


In [110]:
df.head(2) # 데이터확인


,시군구,번지,본번,부번,아파트명,전용면적(㎡),계약년월,계약일,층,건축년도,도로명,target,is_test,번지주소,id,계약년월일,구,동,도로명주소,계약년,번호,k-아파트코드,k-아파트명,"k-단지분류(아파트,주상복합등등)",kapt도로명주소,주소(시군구),주소(읍면동),주소(도로명),k-전체동수,k-전체세대수,k-건설사(시공사),준공일자,k-연면적,k-주거전용면적,k-관리비부과면적,k-전용면적별세대현황(60㎡이하),k-전용면적별세대현황(60㎡~85㎡이하),k-85㎡~135㎡이하,k-135㎡초과,주차대수,좌표X,좌표Y,출입구_x좌표_wgs84,출입구_y좌표_wgs84,지번주소_x,final_X,final_Y,Property_ID,X,Y,Nearest_Bus_Station_Distance,Nearest_Bus_Station_Name,Nearest_Subway_Station_Distance,Nearest_Subway_Station_Name,입주유형,단지명,지번주소_y,동수_x,합계 세대수,분양 세대수,임대 세대수,토지 면적(㎡),합계 동수,5층 동수,5층 세대수,6~10층 동수,6~10층 세대수,11~15층 동수,11~15층 세대수,16~20층 동수,16~20층 세대수,21층이상 동수,21층이상 세대수,~40㎡ 세대수,40㎡~60㎡ 세대수,60㎡~85㎡ 세대수,85㎡~102㎡ 세대수,102㎡~135㎡ 세대수,135㎡~ 세대수,서울특별시_부동산소비심리지수,미분양_구,시점,기준금리,국고채,회사채,국고채-회사채,생산자물가지수,소비자물가지수,서울특별시_가계대출,서울특별시_주택담보대출,서울특별시_주택매매거래량,서울특별시_아파트매매거래량,전국매매가격지수,전국전세가격지수,서울특별시_주택인허가실적,서울특별시_주택착공실적,서울특별시_주택준공실적,서울특별시_주택미분양,미분양_서울특별시,지번주소,공시가격기반단지명,동수_y,세대수,사용승인일자,소형학원수,중형학원수,대형학원수,종합학원수,입시학원수,외국어학원수,예체능학원수,독서실수,정보학원수,특수교육학원수,직업기술학원수,기타학원수,번지_jm,좌표X_jm,좌표Y_jm
0,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.97,201712,8,3,1987,언주로 3,124000.0,0,서울특별시 강남구 개포동 658-1,서울특별시 강남구 개포동 658-1 3층,2017-12-08,강남구,개포동,서울특별시 강남구 언주로 3,2017,40291.0,A10023565,개포6차우성아파트,아파트,서울특별시 강남구 언주로 3,강남구,개포동,언주로,8.0,270.0,우성건설,1987-11-21,22637.0,20204.0,22637.0,20.0,250.0,0.0,NaN,262.0,127.05721,37.476763,127.058091,37.476367,서울특별시 강남구 개포동 658-1,127.05721,37.476763,서울특별시 강남구 언주로 3,127.05721,37.476763,0.061744,개포우성아파트,1.129065,구룡,분양,6차우성,서울특별시 강남구 개포동 658-1,8.0,270.0,270.0,NaN,22539.0,8.0,8.0,270.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0,250.0,NaN,NaN,NaN,94.2,0.0,201712,1.5,2.1,2.649,0.549,99.09,97.698,282438.7,176991.0,13740.0,8484.0,87.701,93.042,113131.0,13157.0,5968.0,45.0,45.0,서울특별시 강남구 개포동 658-1,개포6차우성아파트1동~8동,8.0,270.0,19871121.0,39.0,3.0,0.0,2.0,23.0,1.0,14.0,0.0,0.0,0.0,1.0,1.0,658-1,127.056859,37.476276
1,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.97,201712,22,4,1987,언주로 3,123500.0,0,서울특별시 강남구 개포동 658-1,서울특별시 강남구 개포동 658-1 4층,2017-12-22,강남구,개포동,서울특별시 강남구 언주로 3,2017,40291.0,A10023565,개포6차우성아파트,아파트,서울특별시 강남구 언주로 3,강남구,개포동,언주로,8.0,270.0,우성건설,1987-11-21,22637.0,20204.0,22637.0,20.0,250.0,0.0,NaN,262.0,127.05721,37.476763,127.058091,37.476367,서울특별시 강남구 개포동 658-1,127.05721,37.476763,서울특별시 강남구 언주로 3,127.05721,37.476763,0.061744,개포우성아파트,1.129065,구룡,분양,6차우성,서울특별시 강남구 개포동 658-1,8.0,270.0,270.0,NaN,22539.0,8.0,8.0,270.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0,250.0,NaN,NaN,NaN,94.2,0.0,201712,1.5,2.1,2.649,0.549,99.09,97.698,282438.7,176991.0,13740.0,8484.0,87.701,93.042,113131.0,13157.0,5968.0,45.0,45.0,서울특별시 강남구 개포동 658-1,개포6차우성아파트1동~8동,8.0,270.0,19871121.0,39.0,3.0,0.0,2.0,23.0,1.0,14.0,0.0,0.0,0.0,1.0,1.0,658-1,127.056859,37.476276


### 도로명 데이터 확인

In [113]:
# 열 이름에 '도로명'이 들어간 열들 확인
search_term = '도로명'
column_names = df.columns
found_columns = [col for col in column_names if search_term in col]

In [114]:
found_columns # 확인 -> kapt도로명주소, '주소(도로명)'은 결측치가 있어서 원본파일에 있던 '도로명', '도로명주소' 사용하기로 함

['도로명', '도로명주소', 'kapt도로명주소', '주소(도로명)']

In [ ]:
# 시군구           0
# 번지          227
# 본번           75
# 부번           75
# 아파트명       2136
# 전용면적(㎡)       0
# 계약년월          0
# 계약일           0
# 층             0
# 건축년도          0
# 도로명           0
# target     9272
# is_test       0
# 번지주소        227
# id          227
# 계약년월일         0
# 구             0
# 동             0
# 도로명주소         0
# 계약년           0
# 번호                         263415
# k-아파트코드                    263415
# k-아파트명                     263415
# k-단지분류(아파트,주상복합등등)         263415
# kapt도로명주소                  263415
# 주소(시군구)                    263415
# 주소(읍면동)                    263415
# 주소(도로명)                    263415
# k-전체동수                     263415
# k-전체세대수                    263415
# k-건설사(시공사)                 263865
# 준공일자                       263415
# k-연면적                      263415
# k-주거전용면적                   263476
# k-관리비부과면적                  263415
# k-전용면적별세대현황(60㎡이하)         263476
# k-전용면적별세대현황(60㎡~85㎡이하)     263476
# k-85㎡~135㎡이하               263476
# k-135㎡초과                  1127860
# 주차대수                       266282
# 좌표X                                264006
# 좌표Y                                263936
# 출입구_x좌표_wgs84                      338925
# 출입구_y좌표_wgs84                      338925
# 지번주소_x                             339077
# final_X                             83354
# final_Y                             83284
# Property_ID                             0
# X                                   83631
# Y                                   83561
# Nearest_Bus_Station_Distance        83631
# Nearest_Bus_Station_Name                0
# Nearest_Subway_Station_Distance     83631
# Nearest_Subway_Station_Name             0
# 입주유형                               298650
# 단지명                                298650
# 지번주소_y                             298650
# 동수_x                               298650
# 합계 세대수                             298650
# 분양 세대수                             298872
# 임대 세대수             1063451
# 토지 면적(㎡)            393028
# 합계 동수               298650
# 5층 동수              1086710
# 5층 세대수             1082661
# 6~10층 동수           1023845
# 6~10층 세대수          1023845
# 11~15층 동수           643203
# 11~15층 세대수          643203
# 16~20층 동수           803398
# 16~20층 세대수          803398
# 21층이상 동수            858287
# 21층이상 세대수           858287
# ~40㎡ 세대수           1016871
# 40㎡~60㎡ 세대수         617336
# 60㎡~85㎡ 세대수         461347
# 85㎡~102㎡ 세대수        900811
# 102㎡~135㎡ 세대수       661888
# 135㎡~ 세대수           937574
# 서울특별시_부동산소비심리지수     263829
# 미분양_구               9982
# 시점                     0
# 기준금리                   0
# 국고채                    0
# 회사채                    0
# 국고채-회사채                0
# 생산자물가지수                0
# 소비자물가지수                0
# 서울특별시_가계대출         53544
# 서울특별시_주택담보대출       53544
# 서울특별시_주택매매거래량          0
# 서울특별시_아파트매매거래량         0
# 전국매매가격지수               0
# 전국전세가격지수               0
# 서울특별시_주택인허가실적          0
# 서울특별시_주택착공실적      233727
# 서울특별시_주택준공실적      208820
# 서울특별시_주택미분양            0
# 미분양_서울특별시              0
# 지번주소              344872
# 공시가격기반단지명    344872
# 동수_y         344872
# 세대수          344872
# 사용승인일자       344872
# 소형학원수        344872
# 중형학원수        344872
# 대형학원수        344872
# 종합학원수        344872
# 입시학원수        344872
# 외국어학원수       344872
# 예체능학원수       344872
# 독서실수         344872
# 정보학원수        344872
# 특수교육학원수      344872
# 직업기술학원수      344872
# 기타학원수        344872
# 번지_jm             0
# 좌표X_jm            0
# 좌표Y_jm            0

In [123]:
df.to_csv('trainset_0715_jm.csv', index=False) # 저장

In [231]:
df = pd.read_csv('trainset_0715_jm.csv') # 불러오기

In [232]:
df.shape # shape확인

(1128094, 120)

In [234]:
df[['도로명', '도로명주소', 'kapt도로명주소', '주소(도로명)']].isna().sum()

도로명               0
도로명주소             0
kapt도로명주소    263415
주소(도로명)      263415
dtype: int64

In [235]:
df[['도로명', '도로명주소', 'kapt도로명주소', '주소(도로명)']].head(2)

,도로명,도로명주소,kapt도로명주소,주소(도로명)
0,언주로 3,서울특별시 강남구 언주로 3,서울특별시 강남구 언주로 3,언주로
1,언주로 3,서울특별시 강남구 언주로 3,서울특별시 강남구 언주로 3,언주로


### 도로명주소에서 특정 글자수 이하면 데이터 확인
- 주소는 기본적인 글자수가 있으니 너무 짧은 글자는 이상치로 추정 => 결측치로 변경

In [236]:
selected_indices = []

# 데이터프레임 순회하며 조건을 만족하는 값의 인덱스를 리스트에 추가
for idx, value in enumerate(df['도로명주소']):
    if len(str(value)) <= 10:  # 도로명주소의 글자가 10글자 이하이면 인텍스 추출
        selected_indices.append(idx)

print("조건을 만족하는 값의 인덱스:")
print(selected_indices)

조건을 만족하는 값의 인덱스:
[91517, 91518, 91519, 91520, 91521, 438860]


In [237]:
df.columns.get_loc('도로명') # '도로명' 열도 함께 처리하기 위해 몇 번째 칼럼인지 확인

10

In [238]:
# 데이터확인
df.iloc[selected_indices][['도로명', '도로명주소']] 

,도로명,도로명주소
91517,,서울특별시 중구
91518,,서울특별시 중구
91519,,서울특별시 중구
91520,,서울특별시 중구
91521,,서울특별시 중구
438860,,서울특별시 중구


In [239]:
# 이상치 -> 결측치로 변경
df.iloc[selected_indices, 10] = np.nan
df.iloc[selected_indices, 18] = np.nan

In [240]:
df.iloc[selected_indices][['도로명', '도로명주소']]  # 확인

,도로명,도로명주소
91517,NaN,NaN
91518,NaN,NaN
91519,NaN,NaN
91520,NaN,NaN
91521,NaN,NaN
438860,NaN,NaN


In [241]:
selected_indices = []

# 데이터프레임 순회하며 조건을 만족하는 값의 인덱스를 리스트에 추가
for idx, value in enumerate(df['도로명주소']):
    if len(str(value)) == 11:  
        selected_indices.append(idx)

print("조건을 만족하는 값의 인덱스:")
print(selected_indices)

조건을 만족하는 값의 인덱스:
[63981, 63986, 63987, 133731, 133732, 133735, 172114, 172115, 172116, 172117, 172118, 172119, 172120, 172121, 172122, 181751, 181752, 181753, 181754, 186230, 186231, 186232, 186233, 186234, 186235, 186236, 186237, 186238, 186239, 199957, 215367, 218285, 231402, 234177, 234178, 234179, 234180, 234181, 234182, 234184, 239612, 268537, 268538, 268539, 268540, 268543, 268544, 276862, 278680, 278681, 278691, 280535, 280536, 281363, 282757, 364596, 367408, 367409, 370494, 370496, 370497, 370498, 370499, 370500, 370502, 370503, 370504, 370505, 370506, 370508, 370510, 370511, 370512, 370513, 370514, 370515, 370516, 370518, 370520, 370521, 370522, 370523, 370524, 395419, 403894, 403895, 403896, 424881, 424883, 425375, 425376, 425378, 425380, 427380, 431869, 431870, 431871, 431872, 431873, 431874, 431875, 446905, 446908, 446909, 446910, 446911, 448227, 448243, 448258, 448291, 448298, 448371, 448399, 448404, 448406, 448432, 461825, 495157, 499567, 499568, 508535, 508545, 509850, 5

In [242]:
df.iloc[selected_indices][['도로명', '도로명주소']].value_counts() 

도로명  도로명주소      
     서울특별시 중랑구      735
     서울특별시 송파구       89
     서울특별시 종로구       61
     서울특별시 마포구       44
     서울특별시 관악구       43
     서울특별시 강남구       37
     서울특별시 용산구       29
     서울특별시 성북구       24
     서울특별시 서초구       22
     서울특별시 성동구       18
     서울특별시 강동구       10
     서울특별시 양천구        7
     서울특별시 강서구        6
     서울특별시 광진구        3
     서울특별시 동작구        1
     서울특별시 은평구        1
Name: count, dtype: int64

In [243]:
df.iloc[selected_indices, 10] = np.nan
df.iloc[selected_indices, 18] = np.nan

In [244]:
df.iloc[selected_indices][['도로명', '도로명주소']]

,도로명,도로명주소
63981,NaN,NaN
63986,NaN,NaN
63987,NaN,NaN
133731,NaN,NaN
133732,NaN,NaN
...,...,...
1115595,NaN,NaN
1115601,NaN,NaN
1115602,NaN,NaN
1124312,NaN,NaN


In [245]:
selected_indices = []

# 데이터프레임 순회하며 조건을 만족하는 값의 인덱스를 리스트에 추가
for idx, value in enumerate(df['도로명주소']):
    if len(str(value)) == 12:  
        selected_indices.append(idx)

print("조건을 만족하는 값의 인덱스:")
print(selected_indices)

조건을 만족하는 값의 인덱스:
[82306, 82307, 82308, 193922, 193923, 193924, 193925, 193926, 193927, 193928, 193929, 212543, 212544, 212547, 212549, 230574, 270762, 270763, 351618, 351619, 352721, 352722, 352723, 352724, 352725, 352726, 461338, 501132, 602898, 602901, 602902, 602904, 1000280, 1000281, 1000282, 1000283, 1000284, 1044529, 1067203, 1067204, 1067205, 1067206, 1067207, 1067208, 1067209, 1067210, 1067211, 1067212, 1067213, 1067214, 1067215, 1067216, 1067217, 1067218, 1067219, 1067220, 1067221, 1067222, 1067223, 1067224, 1112121, 1112122, 1112123, 1112124, 1112125, 1112126, 1112127, 1112128, 1112129, 1112130, 1112131, 1112132, 1112133, 1112134, 1112135, 1123728]


In [246]:
df.iloc[selected_indices][['도로명', '도로명주소']].value_counts() 

도로명  도로명주소       
     서울특별시 영등포구      53
     서울특별시 서대문구      21
     서울특별시 동대문구       1
 5   서울특별시 관악구  5     1
Name: count, dtype: int64

In [247]:
df.iloc[selected_indices, 10] = np.nan
df.iloc[selected_indices, 18] = np.nan

In [248]:
df.iloc[selected_indices][['도로명', '도로명주소']]

,도로명,도로명주소
82306,NaN,NaN
82307,NaN,NaN
82308,NaN,NaN
193922,NaN,NaN
193923,NaN,NaN
...,...,...
1112132,NaN,NaN
1112133,NaN,NaN
1112134,NaN,NaN
1112135,NaN,NaN


In [249]:
selected_indices = []

# 데이터프레임 순회하며 조건을 만족하는 값의 인덱스를 리스트에 추가
for idx, value in enumerate(df['도로명주소']):
    if len(str(value)) == 13:  
        selected_indices.append(idx)

print("조건을 만족하는 값의 인덱스:")
print(selected_indices)

조건을 만족하는 값의 인덱스:
[67838, 67842, 67845, 67846, 67847, 67850, 67851, 67854, 231038, 233821, 233893, 249335, 278373, 278374, 278610, 278612, 291001, 347343, 363209, 363210, 363211, 363212, 363213, 363214, 363215, 363216, 363217, 363218, 363219, 363220, 363221, 363222, 363223, 363632, 363633, 363634, 363635, 363636, 363637, 363638, 363639, 363640, 363641, 363642, 363643, 363644, 363645, 363646, 363647, 363648, 363649, 363650, 363651, 363652, 363653, 363655, 363656, 363657, 363658, 363659, 363660, 363661, 363662, 363664, 363665, 363666, 363667, 363668, 363669, 363670, 363672, 363673, 363674, 363703, 426460, 426461, 426632, 426634, 426635, 426636, 426637, 426638, 426639, 426640, 426641, 426667, 445077, 462300, 465715, 472549, 479329, 479330, 479331, 479332, 479340, 479346, 479350, 479351, 479354, 479355, 479360, 505553, 514959, 514960, 514962, 514963, 515382, 515383, 519846, 527444, 527445, 539825, 581039, 728766, 728767, 728768]


In [250]:
df.iloc[selected_indices][['도로명', '도로명주소']].value_counts() 

도로명   도로명주소        
 64   서울특별시 성북구  64    53
 37   서울특별시 성북구  37    35
 20   서울특별시 노원구  20    12
 38   서울특별시 성북구  38     3
 10   서울특별시 양천구  10     2
 27   서울특별시 관악구  27     2
 18   서울특별시 마포구  18     1
 20   서울특별시 강남구  20     1
 27   서울특별시 동작구  27     1
 30   서울특별시 구로구  30     1
 43   서울특별시 종로구  43     1
 50   서울특별시 서초구  50     1
 55   서울특별시 송파구  55     1
 62   서울특별시 광진구  62     1
만리재로  서울특별시 중구 만리재로     1
Name: count, dtype: int64

In [251]:
df.iloc[selected_indices, 10] = np.nan
df.iloc[selected_indices, 18] = np.nan

In [252]:
df.iloc[selected_indices][['도로명', '도로명주소']]

,도로명,도로명주소
67838,NaN,NaN
67842,NaN,NaN
67845,NaN,NaN
67846,NaN,NaN
67847,NaN,NaN
...,...,...
539825,NaN,NaN
581039,NaN,NaN
728766,NaN,NaN
728767,NaN,NaN


In [253]:
selected_indices = []

# 데이터프레임 순회하며 조건을 만족하는 값의 인덱스를 리스트에 추가
for idx, value in enumerate(df['도로명주소']):
    if len(str(value)) == 14:  
        selected_indices.append(idx)

print("조건을 만족하는 값의 인덱스:")
print(selected_indices)

조건을 만족하는 값의 인덱스:
[95372, 95373, 95374, 95375, 95377, 95378, 95379, 95380, 95382, 95383, 95384, 95385, 95387, 95390, 95391, 95395, 95396, 95397, 95399, 95401, 95402, 95404, 95406, 95407, 95408, 95409, 95410, 95412, 95413, 95416, 95418, 95420, 95421, 95424, 95426, 95427, 95429, 95431, 95433, 95434, 95436, 95438, 95441, 95442, 95443, 95444, 95445, 95446, 95447, 95448, 95450, 95452, 95454, 95455, 95456, 95458, 95460, 95463, 95464, 95467, 95468, 95469, 95470, 95471, 95472, 95473, 95474, 95476, 95477, 95478, 95480, 95483, 95485, 95486, 95487, 95488, 95489, 95490, 95491, 95492, 95493, 95495, 95496, 95497, 95498, 95499, 95502, 95503, 95505, 95506, 134016, 134017, 134020, 134023, 134024, 134028, 134029, 134030, 134031, 134035, 134036, 134037, 134038, 134039, 134040, 134041, 134042, 134043, 134044, 134045, 134046, 134047, 134048, 134050, 134051, 134053, 134054, 134055, 134058, 134059, 134063, 134064, 134066, 134067, 134068, 134069, 134071, 134072, 134074, 134075, 134078, 134079, 134080, 134081, 

In [254]:
df.iloc[selected_indices][['도로명', '도로명주소']].value_counts() 

도로명   도로명주소         
개포로   서울특별시 강남구 개포로     544
목동로   서울특별시 양천구 목동로      58
삼개로   서울특별시 마포구 삼개로      53
서운로   서울특별시 서초구 서운로      27
상암로   서울특별시 강동구 상암로      20
노원로   서울특별시 노원구 노원로       4
명일로   서울특별시 강동구 명일로       2
 210  서울특별시 강남구  210      1
 232  서울특별시 노원구  232      1
 245  서울특별시 송파구  245      1
 260  서울특별시 강남구  260      1
 295  서울특별시 도봉구  295      1
 298  서울특별시 송파구  298      1
 348  서울특별시 중랑구  348      1
고덕로   서울특별시 강동구 고덕로       1
Name: count, dtype: int64

In [255]:
df.iloc[selected_indices, 10] = np.nan
df.iloc[selected_indices, 18] = np.nan

In [256]:
df.iloc[selected_indices][['도로명', '도로명주소']]

,도로명,도로명주소
95372,NaN,NaN
95373,NaN,NaN
95374,NaN,NaN
95375,NaN,NaN
95377,NaN,NaN
...,...,...
1110418,NaN,NaN
1110420,NaN,NaN
1110421,NaN,NaN
1110424,NaN,NaN


In [257]:
selected_indices = []

# 데이터프레임 순회하며 조건을 만족하는 값의 인덱스를 리스트에 추가
for idx, value in enumerate(df['도로명주소']):
    if len(str(value)) == 15:  
        selected_indices.append(idx)

print("조건을 만족하는 값의 인덱스:")
print(selected_indices)

조건을 만족하는 값의 인덱스:
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 44, 61, 71, 80, 81, 83, 85, 90, 92, 104, 119, 128, 143, 145, 146, 170, 173, 193, 196, 197, 200, 26509, 26510, 26511, 26512, 26513, 26514, 26515, 26516, 26517, 26518, 26519, 26520, 26521, 28795, 28818, 30748, 41101, 57997, 57998, 79558, 79559, 79560, 79561, 79562, 79563, 79564, 79565, 79566, 79567, 79568, 79569, 79570, 79571, 79572, 79573, 79574, 79575, 79576, 79577, 79578, 79579, 79580, 79581, 79582, 79583, 79584, 79585, 79586, 79587, 79588, 79589, 79590, 79591, 79592, 81060, 81061, 81065, 81066, 81067, 81068, 81069, 81074, 81075, 81078, 81079, 81080, 81082, 81083, 81086, 81087, 81089, 87987, 87988, 87989, 87990, 87991, 87992, 87993, 87994, 87995, 87996, 87997, 88116, 88117, 88673, 90014, 90015, 90016, 90017, 90018, 90037, 90038, 90039, 90040, 90041, 92354, 92355, 92356, 92357, 92358, 92359, 92360, 92361, 92362, 92363, 92364, 92365, 92366, 92367, 92368, 92369, 92370, 92371, 92372, 92373, 92377, 92378, 92379, 92380, 92381, 92382, 9

In [258]:
# 도로명의 값이 15글자 -> 내용이 많아져 보기편하게 데이터프레임으로 만들어서 확인
pd.DataFrame(df.iloc[selected_indices][['도로명', '도로명주소']].value_counts()).reset_index()

,도로명,도로명주소,count
0,다산로 32,서울특별시 중구 다산로 32,2698
1,선릉로 8,서울특별시 강남구 선릉로 8,792
2,청구로 64,서울특별시 중구 청구로 64,471
3,선릉로 7,서울특별시 강남구 선릉로 7,448
4,중림로 10,서울특별시 중구 중림로 10,423
5,신목로 7,서울특별시 양천구 신목로 7,326
6,소공로 35,서울특별시 중구 소공로 35,311
7,신반포로,서울특별시 서초구 신반포로,238
8,신목로 9,서울특별시 양천구 신목로 9,188
9,퇴계로 72,서울특별시 중구 퇴계로 72,152


In [259]:
# 인덱스로 데이터 확인
df_col = df.iloc[selected_indices, 18]
df_col = pd.DataFrame(df_col)
df_col

,도로명주소
0,서울특별시 강남구 언주로 3
1,서울특별시 강남구 언주로 3
2,서울특별시 강남구 언주로 3
3,서울특별시 강남구 언주로 3
4,서울특별시 강남구 언주로 3
...,...
1127861,서울특별시 중구 청구로 64
1127862,서울특별시 중구 청구로 64
1127877,서울특별시 중구 중림로 10
1127878,서울특별시 중구 중림로 10


In [260]:
# 이상한 주소 확인하여 데이터프레임에서 찾아서 결측치로 변경

# '특정 단어'가 포함된 행의 인덱스 찾기
# 찾고자 하는 문장
search_sentences = ['서울특별시 서초구 신반포로', '서울특별시 서초구 고무래로', 
                    '서울특별시 성동구 한림말길', '서울특별시 성동구 독서당로', 
                    '서울특별시 중랑구 봉화산로', '서울특별시 은평구 21-9', '서울특별시 은평구 20-3']

# 각 문장별로 검색하여 인덱스 찾기
indices = []
for sentence in search_sentences:
    indices += df_col[df_col['도로명주소'].str.contains(sentence)].index.tolist()

# 중복 제거를 위해 set으로 변환 후 리스트로 다시 변환
indices = list(set(indices))

print("찾고자 하는 문장이 포함된 행의 인덱스:")
print(indices)

찾고자 하는 문장이 포함된 행의 인덱스:
[1075715, 1075716, 1075717, 1075718, 1075719, 1075720, 1075721, 175114, 1075722, 1056268, 1056269, 175118, 175119, 175120, 1056270, 1056271, 175123, 944660, 175125, 175126, 1056272, 175128, 1056274, 1056275, 1056276, 944668, 990271, 944669, 944670, 1056282, 1106978, 990272, 1106980, 1106981, 990273, 990249, 990251, 990253, 990254, 175152, 175153, 175154, 175155, 175156, 944688, 175158, 175159, 175160, 944689, 175162, 944692, 175164, 944695, 175166, 944696, 944701, 175169, 944703, 944707, 990268, 175173, 175174, 175175, 944709, 175177, 175178, 944712, 175180, 944714, 944716, 944717, 175184, 175185, 175186, 175187, 944721, 175189, 944723, 175191, 175192, 175193, 990282, 175197, 175198, 175199, 175200, 990284, 175202, 944738, 175204, 175205, 175206, 944739, 175208, 944740, 944741, 175211, 175212, 944744, 175214, 175215, 175216, 175217, 175218, 175219, 175220, 175221, 990287, 1056366, 990328, 175225, 175226, 990329, 990330, 990331, 990332, 175231, 175232, 990334, 175

In [261]:
df.iloc[indices][['도로명', '도로명주소']] # 확인

,도로명,도로명주소
1075715,봉화산로,서울특별시 중랑구 봉화산로
1075716,봉화산로,서울특별시 중랑구 봉화산로
1075717,봉화산로,서울특별시 중랑구 봉화산로
1075718,봉화산로,서울특별시 중랑구 봉화산로
1075719,봉화산로,서울특별시 중랑구 봉화산로
...,...,...
1106916,독서당로,서울특별시 성동구 독서당로
944621,신반포로,서울특별시 서초구 신반포로
990263,신반포로,서울특별시 서초구 신반포로
1053176,신반포로,서울특별시 서초구 신반포로


In [264]:
selected_indices = []

# 데이터프레임 순회하며 조건을 만족하는 값의 인덱스를 리스트에 추가
for idx, value in enumerate(df['도로명주소']):
    if len(str(value)) == 16:  
        selected_indices.append(idx)

print("조건을 만족하는 값의 인덱스:")
print(selected_indices)

조건을 만족하는 값의 인덱스:
[975, 976, 977, 978, 979, 980, 981, 982, 983, 984, 985, 1221, 1222, 1223, 2156, 2157, 2158, 2159, 2160, 2161, 2162, 2163, 2164, 2165, 2166, 2167, 2168, 2169, 2170, 2171, 2172, 2173, 2174, 2175, 2176, 2177, 2178, 2179, 2180, 2181, 2182, 2183, 2184, 2185, 2186, 2187, 2188, 2189, 2190, 2191, 2192, 2193, 2194, 2195, 2196, 2197, 2198, 2199, 2200, 2201, 2202, 2203, 2204, 2205, 2206, 2207, 6302, 6303, 6304, 6305, 6306, 6307, 6308, 6309, 6310, 6311, 6312, 6313, 6314, 6315, 6316, 6317, 6318, 6319, 6320, 6321, 6322, 6323, 6324, 6325, 6326, 6327, 6328, 6329, 6330, 6331, 6332, 6333, 6334, 6335, 6336, 6337, 6338, 6339, 6340, 6341, 6342, 6343, 6344, 6345, 6346, 6347, 6348, 6349, 6350, 6351, 6352, 6353, 6354, 6355, 6356, 6357, 6358, 6359, 7815, 7816, 7831, 7832, 7833, 7834, 7835, 7837, 8032, 8033, 8093, 8094, 8095, 8096, 8097, 8098, 8099, 8100, 8101, 8102, 8103, 8104, 8105, 8106, 8107, 8108, 8109, 8110, 8111, 8112, 8113, 8114, 8115, 8116, 8117, 8118, 8119, 8120, 8121, 8122, 8123, 812

In [265]:
view_df = pd.DataFrame(df.iloc[selected_indices][['도로명', '도로명주소']].value_counts()).reset_index()
view_df[400:]

,도로명,도로명주소,count
400,장문로 63,서울특별시 용산구 장문로 63,5
401,중앙로 85,서울특별시 구로구 중앙로 85,5
402,장문로 24,서울특별시 용산구 장문로 24,5
403,삼양로 14,서울특별시 성북구 삼양로 14,5
404,도당로 41,서울특별시 도봉구 도당로 41,5
405,은평로 42,서울특별시 은평구 은평로 42,5
406,오목로 36,서울특별시 양천구 오목로 36,5
407,청구로1길 6,서울특별시 중구 청구로1길 6,4
408,이문로 7,서울특별시 동대문구 이문로 7,4
409,관천로 74,서울특별시 관악구 관천로 74,4


In [266]:
df_col = df.iloc[selected_indices, 18]
df_col = pd.DataFrame(df_col)
df_col

,도로명주소
975,서울특별시 강남구 삼성로 14
976,서울특별시 강남구 삼성로 14
977,서울특별시 강남구 삼성로 14
978,서울특별시 강남구 삼성로 14
979,서울특별시 강남구 삼성로 14
...,...
1127883,서울특별시 중구 난계로 169
1127963,서울특별시 중랑구 중랑천로 8
1127965,서울특별시 중랑구 공릉로 32
1128035,서울특별시 중랑구 신내로 51


In [267]:
# '특정 단어'가 포함된 행의 인덱스 찾기
# 찾고자 하는 문장
search_sentences = [' 174-2']

# 각 문장별로 검색하여 인덱스 찾기
indices = []
for sentence in search_sentences:
    indices += df_col[df_col['도로명주소'].str.contains(sentence)].index.tolist()

# 중복 제거를 위해 set으로 변환 후 리스트로 다시 변환
indices = list(set(indices))

print("찾고자 하는 문장이 포함된 행의 인덱스:")
print(indices)

찾고자 하는 문장이 포함된 행의 인덱스:
[87815]


In [268]:
df.iloc[indices][['도로명', '도로명주소']]

,도로명,도로명주소
87815,174-2,서울특별시 용산구 174-2


In [269]:
df.iloc[indices, 10] = np.nan
df.iloc[indices, 18] = np.nan

In [270]:
df.iloc[indices][['도로명', '도로명주소']]

,도로명,도로명주소
87815,NaN,NaN


In [ ]:
df_col = df.iloc[selected_indices, 18]
df_col = pd.DataFrame(df_col)
df_col

,도로명주소
975,서울특별시 강남구 삼성로 14
976,서울특별시 강남구 삼성로 14
977,서울특별시 강남구 삼성로 14
978,서울특별시 강남구 삼성로 14
979,서울특별시 강남구 삼성로 14
...,...
1127883,서울특별시 중구 난계로 169
1127963,서울특별시 중랑구 중랑천로 8
1127965,서울특별시 중랑구 공릉로 32
1128035,서울특별시 중랑구 신내로 51


In [ ]:
# '특정 단어'가 포함된 행의 인덱스 찾기
# 찾고자 하는 문장
search_sentences = [' 174-2']

# 각 문장별로 검색하여 인덱스 찾기
indices = []
for sentence in search_sentences:
    indices += df_col[df_col['도로명주소'].str.contains(sentence)].index.tolist()

# 중복 제거를 위해 set으로 변환 후 리스트로 다시 변환
indices = list(set(indices))

print("찾고자 하는 문장이 포함된 행의 인덱스:")
print(indices)

찾고자 하는 문장이 포함된 행의 인덱스:
[87815]


In [ ]:
df.iloc[indices][['도로명', '도로명주소']]

,도로명,도로명주소
87815,174-2,서울특별시 용산구 174-2


In [ ]:
df.iloc[indices, 10] = np.nan
df.iloc[indices, 18] = np.nan

In [ ]:
df.iloc[indices][['도로명', '도로명주소']]

,도로명,도로명주소
87815,NaN,NaN


In [282]:
df['도로명주소'].isna().sum()

2336

In [304]:
selected_indices = []

# 데이터프레임 순회하며 조건을 만족하는 값의 인덱스를 리스트에 추가
for idx, value in enumerate(df['도로명']):
    if len(str(value)) == 5:  
        selected_indices.append(idx)

print("조건을 만족하는 값의 인덱스:")
print(selected_indices)

조건을 만족하는 값의 인덱스:
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 44, 61, 71, 80, 81, 83, 85, 90, 92, 104, 119, 128, 143, 145, 146, 170, 173, 193, 196, 197, 200, 26509, 26510, 26511, 26512, 26513, 26514, 26515, 26516, 26517, 26518, 26519, 26520, 26521, 28795, 28818, 30748, 41101, 46302, 46303, 46304, 46305, 46306, 55670, 55671, 55672, 57997, 57998, 79558, 79559, 79560, 79561, 79562, 79563, 79564, 79565, 79566, 79567, 79568, 79569, 79570, 79571, 79572, 79573, 79574, 79575, 79576, 79577, 79578, 79579, 79580, 79581, 79582, 79583, 79584, 79585, 79586, 79587, 79588, 79589, 79590, 79591, 79592, 81060, 81061, 81065, 81066, 81067, 81068, 81069, 81074, 81075, 81078, 81079, 81080, 81082, 81083, 81086, 81087, 81089, 85150, 85151, 85152, 85153, 85154, 85155, 85156, 85157, 85158, 85159, 85160, 85161, 85162, 85163, 85164, 85165, 85166, 87987, 87988, 87989, 87990, 87991, 87992, 87993, 87994, 87995, 87996, 87997, 88116, 88117, 88673, 90014, 90015, 90016, 90017, 90018, 90037, 90038, 90039, 90040, 90041, 95369, 9

In [313]:
view_df = pd.DataFrame(df.iloc[selected_indices][['도로명', '도로명주소']].value_counts()).reset_index()
view_df.iloc[-2, 1]

'서울특별시 은평구  21-9'

In [310]:
df_col = df.iloc[selected_indices, 18]
df_col = pd.DataFrame(df_col)
df_col

,도로명주소
0,서울특별시 강남구 언주로 3
1,서울특별시 강남구 언주로 3
2,서울특별시 강남구 언주로 3
3,서울특별시 강남구 언주로 3
4,서울특별시 강남구 언주로 3
...,...
1123276,서울특별시 동작구 만양로 6
1123277,서울특별시 동작구 만양로 6
1123278,서울특별시 동작구 만양로 6
1123279,서울특별시 동작구 만양로 6


In [314]:
# '특정 단어'가 포함된 행의 인덱스 찾기
# 찾고자 하는 문장
search_sentences = ['서울특별시 은평구  21-9','서울특별시 은평구  20-3']

# 각 문장별로 검색하여 인덱스 찾기
indices = []
for sentence in search_sentences:
    indices += df_col[df_col['도로명주소'].str.contains(sentence)].index.tolist()

# 중복 제거를 위해 set으로 변환 후 리스트로 다시 변환
indices = list(set(indices))

print("찾고자 하는 문장이 포함된 행의 인덱스:")
print(indices)

찾고자 하는 문장이 포함된 행의 인덱스:
[538868, 538661]


In [315]:
df.iloc[indices][['도로명', '도로명주소']]

,도로명,도로명주소
538868,21-9,서울특별시 은평구 21-9
538661,20-3,서울특별시 은평구 20-3


In [316]:
df.iloc[indices, 10] = np.nan
df.iloc[indices, 18] = np.nan

In [317]:
df.iloc[indices][['도로명', '도로명주소']]

,도로명,도로명주소
538868,NaN,NaN
538661,NaN,NaN


In [318]:
selected_indices = []

# 데이터프레임 순회하며 조건을 만족하는 값의 인덱스를 리스트에 추가
for idx, value in enumerate(df['도로명']):
    if len(str(value)) == 6:  
        selected_indices.append(idx)

print("조건을 만족하는 값의 인덱스:")
print(selected_indices)

조건을 만족하는 값의 인덱스:
[975, 976, 977, 978, 979, 980, 981, 982, 983, 984, 985, 1221, 1222, 1223, 2156, 2157, 2158, 2159, 2160, 2161, 2162, 2163, 2164, 2165, 2166, 2167, 2168, 2169, 2170, 2171, 2172, 2173, 2174, 2175, 2176, 2177, 2178, 2179, 2180, 2181, 2182, 2183, 2184, 2185, 2186, 2187, 2188, 2189, 2190, 2191, 2192, 2193, 2194, 2195, 2196, 2197, 2198, 2199, 2200, 2201, 2202, 2203, 2204, 2205, 2206, 2207, 6302, 6303, 6304, 6305, 6306, 6307, 6308, 6309, 6310, 6311, 6312, 6313, 6314, 6315, 6316, 6317, 6318, 6319, 6320, 6321, 6322, 6323, 6324, 6325, 6326, 6327, 6328, 6329, 6330, 6331, 6332, 6333, 6334, 6335, 6336, 6337, 6338, 6339, 6340, 6341, 6342, 6343, 6344, 6345, 6346, 6347, 6348, 6349, 6350, 6351, 6352, 6353, 6354, 6355, 6356, 6357, 6358, 6359, 7815, 7816, 7831, 7832, 7833, 7834, 7835, 7837, 8032, 8033, 8093, 8094, 8095, 8096, 8097, 8098, 8099, 8100, 8101, 8102, 8103, 8104, 8105, 8106, 8107, 8108, 8109, 8110, 8111, 8112, 8113, 8114, 8115, 8116, 8117, 8118, 8119, 8120, 8121, 8122, 8123, 812

In [332]:
view_df = pd.DataFrame(df.iloc[selected_indices][['도로명', '도로명주소']].value_counts()).reset_index()
view_df.iloc[-1, 1]

'서울특별시 서대문구  35-28'

In [333]:
df_col = df.iloc[selected_indices, 18]
df_col = pd.DataFrame(df_col)
df_col

,도로명주소
975,서울특별시 강남구 삼성로 14
976,서울특별시 강남구 삼성로 14
977,서울특별시 강남구 삼성로 14
978,서울특별시 강남구 삼성로 14
979,서울특별시 강남구 삼성로 14
...,...
1127879,서울특별시 중구 중림로 10
1127963,서울특별시 중랑구 중랑천로 8
1127965,서울특별시 중랑구 공릉로 32
1128035,서울특별시 중랑구 신내로 51


In [334]:
# '특정 단어'가 포함된 행의 인덱스 찾기
# 찾고자 하는 문장
search_sentences = ['서울특별시 서대문구  35-28']

# 각 문장별로 검색하여 인덱스 찾기
indices = []
for sentence in search_sentences:
    indices += df_col[df_col['도로명주소'].str.contains(sentence)].index.tolist()

# 중복 제거를 위해 set으로 변환 후 리스트로 다시 변환
indices = list(set(indices))

print("찾고자 하는 문장이 포함된 행의 인덱스:")
print(indices)

찾고자 하는 문장이 포함된 행의 인덱스:
[502544]


In [335]:
df.iloc[indices][['도로명', '도로명주소']]

,도로명,도로명주소
502544,35-28,서울특별시 서대문구 35-28


In [336]:
df.iloc[indices, 10] = np.nan
df.iloc[indices, 18] = np.nan

In [337]:
df.iloc[indices][['도로명', '도로명주소']]

,도로명,도로명주소
502544,NaN,NaN


In [338]:
df['도로명주소'].isna().sum()

2339

In [ ]:
['도로명', '도로명주소']

### 저장

In [339]:
df.to_csv('train_240716.csv', index=False)

In [346]:
df = pd.read_csv('../data/train_240716.csv')

In [347]:
df_2 = pd.read_csv('../data/trainset_0715.csv')

### 열 추가

In [349]:
df[['도로명_nan', '도로명주소_nan']] = df[['도로명', '도로명주소']]

In [352]:
df[['도로명', '도로명주소']] = df_2[['도로명', '도로명주소']]

In [354]:
df[['도로명', '도로명주소']].isna().sum()

도로명      0
도로명주소    0
dtype: int64

In [356]:
df[['도로명_nan', '도로명주소_nan']].isna().sum()

도로명_nan      2339
도로명주소_nan    2339
dtype: int64

### 확인해야되는 행이 1000개 넘어가면 보는 의미가 없을 듯해서 스톱!

### 다시저장

In [357]:
df.to_csv('trainset_240716.csv', index=False)

In [371]:
df[['시군구','번지_jm']].isna().sum()

시군구      0
번지_jm    0
dtype: int64

In [373]:
df['번지주소_2'].isna().sum()

0

## 도로명주소 결측치 더 메꾸기
- 번지주소_2 기준으로 주소 API사용해서 도로명주소 추출

In [395]:
addr = df[df['도로명주소_nan'].isna()]['번지주소_2']
addr = addr.unique()
addr = pd.DataFrame(addr).reset_index(drop=True)

In [397]:
addr.columns = ['번지주소_2']
addr

,번지주소_2
0,서울특별시 성동구 하왕십리동 1071
1,서울특별시 성북구 보문동4가 116
2,서울특별시 영등포구 당산동4가 99
3,서울특별시 용산구 한강로1가 231-23
4,서울특별시 중구 만리동2가 176-1
...,...
85,서울특별시 중랑구 상봉동 484
86,서울특별시 강남구 대치동 633
87,서울특별시 강남구 역삼동 763
88,서울특별시 서초구 서초동 1686-4


In [378]:
from PyKakao import Local

#### API 키 -> 사용시 추가하면 됨

In [379]:
api = Local(service_key = "api키")

In [380]:
# api 잘 돌아가는지 확인
ans =  api.search_address("서울특별시 성북구 보문동4가 116")
ans

{'documents': [{'address': {'address_name': '서울 성북구 보문동4가 116',
    'b_code': '1129012600',
    'h_code': '1129061000',
    'main_address_no': '116',
    'mountain_yn': 'N',
    'region_1depth_name': '서울',
    'region_2depth_name': '성북구',
    'region_3depth_h_name': '보문동',
    'region_3depth_name': '보문동4가',
    'sub_address_no': '',
    'x': '127.020967748216',
    'y': '37.5841413862939'},
   'address_name': '서울 성북구 보문동4가 116',
   'address_type': 'REGION_ADDR',
   'road_address': {'address_name': '서울 성북구 고려대로2길 37',
    'building_name': '보문하우스토리',
    'main_building_no': '37',
    'region_1depth_name': '서울',
    'region_2depth_name': '성북구',
    'region_3depth_name': '보문동4가',
    'road_name': '고려대로2길',
    'sub_building_no': '',
    'underground_yn': 'N',
    'x': '127.020855841499',
    'y': '37.5843732316748',
    'zone_no': '02852'},
   'x': '127.020967748216',
   'y': '37.5841413862939'}],
 'meta': {'is_end': True, 'pageable_count': 1, 'total_count': 1}}

In [398]:
results = []

# 데이터프레임 순회하면서 API 호출 및 결과 저장
for index, data in addr.iterrows():
    road_address = data['번지주소_2']
    # API 호출 (여기서는 예시로 함수 이름이 api_function이라 가정)
    api_response = api.search_address(road_address)
    
    if api_response is None:
        print(f"API did not return a valid response for address: {road_address}")
        continue
    
    # API 응답에서 필요한 데이터 추출
    documents = api_response.get('documents', [])
    for doc in documents:
        address_info = doc.get('address', {})
        road_address_info = doc.get('road_address') or {}

        result_dict = {
            'address_name': address_info.get('address_name', ''),
            'b_code': address_info.get('b_code', ''),
            'h_code': address_info.get('h_code', ''),
            'main_address_no': address_info.get('main_address_no', ''),
            'mountain_yn': address_info.get('mountain_yn', ''),
            'region_1depth_name': address_info.get('region_1depth_name', ''),
            'region_2depth_name': address_info.get('region_2depth_name', ''),
            'region_3depth_h_name': address_info.get('region_3depth_h_name', ''),
            'region_3depth_name': address_info.get('region_3depth_name', ''),
            'sub_address_no': address_info.get('sub_address_no', ''),
            'x': address_info.get('x', ''),
            'y': address_info.get('y', ''),
            'road_address_name': road_address_info.get('address_name', ''),
            'building_name': road_address_info.get('building_name', ''),
            'main_building_no': road_address_info.get('main_building_no', ''),
            'road_region_1depth_name': road_address_info.get('region_1depth_name', ''),
            'road_region_2depth_name': road_address_info.get('region_2depth_name', ''),
            'road_region_3depth_name': road_address_info.get('region_3depth_name', ''),
            'road_name': road_address_info.get('road_name', ''),
            'sub_building_no': road_address_info.get('sub_building_no', ''),
            'underground_yn': road_address_info.get('underground_yn', ''),
            'road_x': road_address_info.get('x', ''),
            'road_y': road_address_info.get('y', ''),
            'zone_no': road_address_info.get('zone_no', ''),
            'document_x': doc.get('x', ''),
            'document_y': doc.get('y', '')
        }
        results.append(result_dict)

# 결과 리스트를 데이터프레임으로 변환
result_df = pd.DataFrame(results)

# 결과 출력
result_df


,address_name,b_code,h_code,main_address_no,mountain_yn,region_1depth_name,region_2depth_name,region_3depth_h_name,region_3depth_name,sub_address_no,x,y,road_address_name,building_name,main_building_no,road_region_1depth_name,road_region_2depth_name,road_region_3depth_name,road_name,sub_building_no,underground_yn,road_x,road_y,zone_no,document_x,document_y
0,서울 성동구 하왕십리동 1071,1120010200,1120052000,1071,N,서울,성동구,왕십리2동,하왕십리동,,127.024489937553,37.560680599542,서울 성동구 난계로 100,왕십리자이,100,서울,성동구,하왕십리동,난계로,,N,127.024651500492,37.5615293048656,04711,127.024489937553,37.560680599542
1,서울 성북구 보문동4가 116,1129012600,1129061000,116,N,서울,성북구,보문동,보문동4가,,127.020967748216,37.5841413862939,서울 성북구 고려대로2길 37,보문하우스토리,37,서울,성북구,보문동4가,고려대로2길,,N,127.020855841499,37.5843732316748,02852,127.020967748216,37.5841413862939
2,서울 영등포구 당산동4가 99,1156011400,1156056000,99,N,서울,영등포구,당산2동,당산동4가,,126.902942298813,37.5286609363432,서울 영등포구 영신로 237,당산 롯데캐슬 프레스티지,237,서울,영등포구,당산동4가,영신로,,N,126.902807748097,37.5287757033687,07216,126.902942298813,37.5286609363432
3,서울 용산구 한강로1가 231-23,1117012400,1117062500,231,N,서울,용산구,한강로동,한강로1가,23,126.973295518526,37.5338273797012,서울 용산구 한강대로 174-2,,174,서울,용산구,한강로1가,한강대로,2,N,126.973109190446,37.5341017822966,04382,126.973295518526,37.5338273797012
4,서울 중구 만리동2가 288,1114017400,1114068000,288,N,서울,중구,중림동,만리동2가,,126.965563662247,37.5550897722338,서울 중구 만리재로 177,서울역한라비발디센트럴,177,서울,중구,만리동2가,만리재로,,N,126.965847056196,37.5545735828657,04501,126.965563662247,37.5550897722338
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,서울 성동구 옥수동 436,1120011300,1120064500,436,N,서울,성동구,옥수동,옥수동,,127.009867068046,37.542081527886,서울 성동구 독서당로 173,극동그린아파트,173,서울,성동구,옥수동,독서당로,,N,127.010900872837,37.5415625512366,04732,127.009867068046,37.542081527886
67,서울 성동구 옥수동 100,1120011300,1120064500,100,N,서울,성동구,옥수동,옥수동,,127.015956898092,37.539617167963,서울 성동구 한림말길 48,옥수하이츠아파트,48,서울,성동구,옥수동,한림말길,,N,127.016196963074,37.5410102614044,04736,127.015956898092,37.539617167963
68,서울 양천구 목동 925,1147010200,1147051000,925,N,서울,양천구,목1동,목동,,126.867840519363,37.5288232909409,서울 양천구 목동로 186,목동신시가지아파트7단지,186,서울,양천구,목동,목동로,,N,126.866390006625,37.5252268436522,07993,126.867840519363,37.5288232909409
69,서울 종로구 옥인동 179-3,1111011100,1111051500,179,N,서울,종로구,청운효자동,옥인동,3,126.963645256089,37.5825996546239,,,,,,,,,,,,,126.963645256089,37.5825996546239


In [400]:
# result_df.columns 
# 칼럼 확인 후 이름 변경
result_df.columns = ['번지주소_2', 'b_code', 'h_code', 'main_address_no', 'mountain_yn',
       'region_1depth_name', 'region_2depth_name', 'region_3depth_h_name',
       'region_3depth_name', 'sub_address_no', 'x', 'y', 'road_address_name',
       'building_name', 'main_building_no', 'road_region_1depth_name',
       'road_region_2depth_name', 'road_region_3depth_name', 'road_name',
       'sub_building_no', 'underground_yn', 'road_x', 'road_y', 'zone_no',
       'document_x', 'document_y']

In [403]:
# 필요한 열만 추출
result_df = result_df[['번지주소_2', 'road_address_name']]

In [429]:
# '서울 ' -> '서울특별시 ' 로 변환
result_df = result_df.applymap(lambda x: x.replace('서울 ', '서울특별시 ') if isinstance(x, str) else x)

In [430]:
# 병합
addr_df = addr.merge(result_df, on='번지주소_2', how='outer')

In [433]:
addr_df[:20]

,번지주소_2,road_address_name
0,서울특별시 성동구 하왕십리동 1071,서울특별시 성동구 난계로 100
1,서울특별시 성북구 보문동4가 116,서울특별시 성북구 고려대로2길 37
2,서울특별시 영등포구 당산동4가 99,서울특별시 영등포구 영신로 237
3,서울특별시 용산구 한강로1가 231-23,서울특별시 용산구 한강대로 174-2
4,서울특별시 중구 만리동2가 176-1,NaN
5,서울특별시 중구 만리동2가 288,서울특별시 중구 만리재로 177
6,서울특별시 강남구 개포동 141,
7,서울특별시 서초구 반포동 757,
8,서울특별시 서초구 반포동 30-18,NaN
9,서울특별시 서초구 반포동 968,


In [436]:
# 공란은 결측치로 변경
addr_df['road_address_name'].replace('', np.nan, inplace=True)

In [447]:
# addr_df.to_excel('../data/addr.xlsx', index=False)

In [452]:
# 잘 돌아갔는지 주소 검색해서 확인하기
df[df['번지주소_2'].str.contains('서울특별시 강남구 역삼동 763')]

,시군구,번지,본번,부번,아파트명,전용면적(㎡),계약년월,계약일,층,건축년도,도로명,target,is_test,번지주소,id,계약년월일,구,동,도로명주소,계약년,번호,k-아파트코드,k-아파트명,"k-단지분류(아파트,주상복합등등)",kapt도로명주소,주소(시군구),주소(읍면동),주소(도로명),k-전체동수,k-전체세대수,k-건설사(시공사),준공일자,k-연면적,k-주거전용면적,k-관리비부과면적,k-전용면적별세대현황(60㎡이하),k-전용면적별세대현황(60㎡~85㎡이하),k-85㎡~135㎡이하,k-135㎡초과,주차대수,좌표X,좌표Y,출입구_x좌표_wgs84,출입구_y좌표_wgs84,지번주소_x,final_X,final_Y,Property_ID,X,Y,Nearest_Bus_Station_Distance,Nearest_Bus_Station_Name,Nearest_Subway_Station_Distance,Nearest_Subway_Station_Name,입주유형,단지명,지번주소_y,동수_x,합계 세대수,분양 세대수,...,합계 동수,5층 동수,5층 세대수,6~10층 동수,6~10층 세대수,11~15층 동수,11~15층 세대수,16~20층 동수,16~20층 세대수,21층이상 동수,21층이상 세대수,~40㎡ 세대수,40㎡~60㎡ 세대수,60㎡~85㎡ 세대수,85㎡~102㎡ 세대수,102㎡~135㎡ 세대수,135㎡~ 세대수,서울특별시_부동산소비심리지수,미분양_구,시점,기준금리,국고채,회사채,국고채-회사채,생산자물가지수,소비자물가지수,서울특별시_가계대출,서울특별시_주택담보대출,서울특별시_주택매매거래량,서울특별시_아파트매매거래량,전국매매가격지수,전국전세가격지수,서울특별시_주택인허가실적,서울특별시_주택착공실적,서울특별시_주택준공실적,서울특별시_주택미분양,미분양_서울특별시,지번주소,공시가격기반단지명,동수_y,세대수,사용승인일자,소형학원수,중형학원수,대형학원수,종합학원수,입시학원수,외국어학원수,예체능학원수,독서실수,정보학원수,특수교육학원수,직업기술학원수,기타학원수,번지_jm,좌표X_jm,좌표Y_jm,도로명_nan,도로명주소_nan,번지주소_2
2316,서울특별시 강남구 역삼동,763-16,763.0,16.0,래미안그레이튼,84.97,201710,11,4,2009,도곡로43길 20,80000.0,0,서울특별시 강남구 역삼동 763-16,서울특별시 강남구 역삼동 763-16 4층,2017-10-11,강남구,역삼동,서울특별시 강남구 도곡로43길 20,2017,1992.0,A13579505,래미안그레이튼3차,아파트,서울특별시 강남구 도곡로43길 20,강남구,역삼동,도곡로43길,8.0,476.0,삼성물산,2009-12-28,76688.0,39855.0,52944.0,121.0,281.0,0.0,NaN,582.0,127.048158,37.496481,127.047334,37.495865,서울특별시 강남구 역삼동 763-16,127.048158,37.496481,서울특별시 강남구 도곡로43길 20,127.048158,37.496481,0.192683,역삼2동주민센터.대림역삼아파트,0.416592,한티,분양,래미안그레이튼3차,서울특별시 강남구 역삼동 763-16,8.0,476.0,476.0,...,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,476.0,NaN,121.0,281.0,NaN,NaN,74.0,105.6,0.0,201710,1.25,2.026,2.574,0.548,99.14,98.077,278568.7,175427.9,8561.0,3942.0,87.569,93.070,89283.0,4269.0,3299.0,56.0,56.0,서울특별시 강남구 역삼동 763-16,래미안그레이튼,8.0,476.0,20091228.0,340.0,77.0,5.0,10.0,269.0,29.0,75.0,11.0,1.0,0.0,3.0,24.0,763-16,127.048166,37.496507,도곡로43길 20,서울특별시 강남구 도곡로43길 20,서울특별시 강남구 역삼동 763-16
2317,서울특별시 강남구 역삼동,763-16,763.0,16.0,래미안그레이튼,142.06,201710,14,15,2009,도곡로43길 20,179500.0,0,서울특별시 강남구 역삼동 763-16,서울특별시 강남구 역삼동 763-16 15층,2017-10-14,강남구,역삼동,서울특별시 강남구 도곡로43길 20,2017,1992.0,A13579505,래미안그레이튼3차,아파트,서울특별시 강남구 도곡로43길 20,강남구,역삼동,도곡로43길,8.0,476.0,삼성물산,2009-12-28,76688.0,39855.0,52944.0,121.0,281.0,0.0,NaN,582.0,127.048158,37.496481,127.047334,37.495865,서울특별시 강남구 역삼동 763-16,127.048158,37.496481,서울특별시 강남구 도곡로43길 20,127.048158,37.496481,0.192683,역삼2동주민센터.대림역삼아파트,0.416592,한티,분양,래미안그레이튼3차,서울특별시 강남구 역삼동 763-16,8.0,476.0,476.0,...,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,476.0,NaN,121.0,281.0,NaN,NaN,74.0,105.6,0.0,201710,1.25,2.026,2.574,0.548,99.14,98.077,278568.7,175427.9,8561.0,3942.0,87.569,93.070,89283.0,4269.0,3299.0,56.0,56.0,서울특별시 강남구 역삼동 763-16,래미안그레이튼,8.0,476.0,20091228.0,340.0,77.0,5.0,10.0,269.0,29.0,75.0,11.0,1.0,0.0,3.0,24.0,763-16,127.048166,37.496507,도곡로43길 20,서울특별시 강남구 도곡로43길 20,서울특별시 강남구 역삼동 763-16
2318,서울특별시 강남구 역삼동,763-16,763.0,16.0,래미안그레이튼,84.97,201710,24,6,2009,도곡로43길 20,135000.0,0,서울특별시 강남구 역삼동 763-16,서울특별시 강남구 역삼동 763-16 6층,2017-10-24,강남구,역삼동,서울특별시 강남구 도곡로43길 20,2017,1992.0,A13579505,래미안그레이튼3차,아파트,서울특별시 강남구 도곡로43길 20,강남구,역삼동,도곡로43길,8.0,476.0,삼성물산,2009-12-28,76688.0,39855.0,52944.0,121.0,281.0,0.0,NaN,582.0,127.048158,37.496481,127.047334,37.495865,서울특별시 강남구 역삼동 763-16,127.048158,37.496481,서울특별시 강남구 도곡로43길 20,127.048158,37.496481,0.192683,역삼2동주민센터.대림역삼아파트,0.416592,한티,분양,래미안그레이튼3차,서울특별시 강남구 역삼동 763-16,8.0,476.0,476.0,...,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,476.0,NaN,121.0,281.0,NaN,NaN,74.0,105.6,0.0,201710,1.25,2.026,2.574,0.548,99.14,98.077,278568.7,175427.9,8561.0,3942.0,87.569,93.070,89283.0,4269.0,3299.0,56.0,56.0,서울특별시 강남구 역삼동 763-16,래미안그레이튼,8.0,476.0,20091228.0,340.0,77.0,5.0,10.0,269.0,29.0,75.0,11.0,1.0,0.0,3.0,24.0,763-16,127.048166,37.496507,도곡로43길 20,서울특별시 강남구 도곡로43길 20,서울특별시 강남구 역삼동 763-16
2319,서울특별시 강남구 역삼동,763-16,763.0,16.0,래미안그레이튼,84.97,201710,28,9,2009,도곡로43길 20,135000.0,0,서울특별시 강남구 역삼동 763-16,서울특별시 강남구 역삼동 763-16 9층,2017-10-28,강남구,역삼동,서울특별시 강남구 도곡로43길 20,20

In [ ]:
# 주소 결측값있는 년도 확인필요

In [456]:
result_df[:20]

,번지주소_2,road_address_name
0,서울특별시 성동구 하왕십리동 1071,서울특별시 성동구 난계로 100
1,서울특별시 성북구 보문동4가 116,서울특별시 성북구 고려대로2길 37
2,서울특별시 영등포구 당산동4가 99,서울특별시 영등포구 영신로 237
3,서울특별시 용산구 한강로1가 231-23,서울특별시 용산구 한강대로 174-2
4,서울특별시 중구 만리동2가 288,서울특별시 중구 만리재로 177
5,서울특별시 강남구 개포동 141,
6,서울특별시 서초구 반포동 757,
7,서울특별시 서초구 반포동 968,
8,서울특별시 종로구 평창동 596,서울특별시 종로구 평창문화로 170
9,서울특별시 마포구 아현동 372-29,


In [457]:
result_df['road_address_name'].replace('', np.nan, inplace=True)

In [460]:
result_df_2 = result_df.dropna() # 결측치 row 삭제

In [462]:
result_df_2.columns = ['번지주소_2', "도로명주소_추가"] # 열 이름 변경

In [517]:
result_df_2 # 확인

,번지주소_2,도로명주소_추가
0,서울특별시 성동구 하왕십리동 1071,서울특별시 성동구 난계로 100
1,서울특별시 성북구 보문동4가 116,서울특별시 성북구 고려대로2길 37
2,서울특별시 영등포구 당산동4가 99,서울특별시 영등포구 영신로 237
3,서울특별시 용산구 한강로1가 231-23,서울특별시 용산구 한강대로 174-2
4,서울특별시 중구 만리동2가 288,서울특별시 중구 만리재로 177
...,...,...
65,서울특별시 서초구 서초동 1312-3,서울특별시 서초구 서운로 197
66,서울특별시 성동구 옥수동 436,서울특별시 성동구 독서당로 173
67,서울특별시 성동구 옥수동 100,서울특별시 성동구 한림말길 48
68,서울특별시 양천구 목동 925,서울특별시 양천구 목동로 186


In [509]:
df2 = df.merge(result_df_2, on='번지주소_2', how='outer' )

In [502]:
df2.head(2) 
df2['도로명주소_nan'].fillna(df2['도로명주소_추가'], inplace=True)

In [510]:
df2['도로명주소_nan'].isna().sum()

2339

In [518]:
df[['도로명주소', '도로명주소_nan']][100000:]

,도로명주소,도로명주소_nan
100000,서울특별시 강동구 고덕로79길 18,서울특별시 강동구 고덕로79길 18
100001,서울특별시 강동구 고덕로 313,서울특별시 강동구 고덕로 313
100002,서울특별시 강동구 고덕로 313,서울특별시 강동구 고덕로 313
100003,서울특별시 강동구 고덕로79길 18,서울특별시 강동구 고덕로79길 18
100004,서울특별시 강동구 고덕로 313,서울특별시 강동구 고덕로 313
...,...,...
1128089,서울특별시 중랑구 신내역로1길 85,서울특별시 중랑구 신내역로1길 85
1128090,서울특별시 중랑구 신내역로1길 85,서울특별시 중랑구 신내역로1길 85
1128091,서울특별시 중랑구 신내역로1길 85,서울특별시 중랑구 신내역로1길 85
1128092,서울특별시 중랑구 신내역로1길 85,서울특별시 중랑구 신내역로1길 85


In [512]:
df2['도로명_df2'] = df2['도로명주소_nan']
df2['도로명_df'] = df['도로명주소_nan']
df2.head(2)

,시군구,번지,본번,부번,아파트명,전용면적(㎡),계약년월,계약일,층,건축년도,도로명,target,is_test,번지주소,id,계약년월일,구,동,도로명주소,계약년,번호,k-아파트코드,k-아파트명,"k-단지분류(아파트,주상복합등등)",kapt도로명주소,주소(시군구),주소(읍면동),주소(도로명),k-전체동수,k-전체세대수,k-건설사(시공사),준공일자,k-연면적,k-주거전용면적,k-관리비부과면적,k-전용면적별세대현황(60㎡이하),k-전용면적별세대현황(60㎡~85㎡이하),k-85㎡~135㎡이하,k-135㎡초과,주차대수,좌표X,좌표Y,출입구_x좌표_wgs84,출입구_y좌표_wgs84,지번주소_x,final_X,final_Y,Property_ID,X,Y,Nearest_Bus_Station_Distance,Nearest_Bus_Station_Name,Nearest_Subway_Station_Distance,Nearest_Subway_Station_Name,입주유형,단지명,지번주소_y,동수_x,합계 세대수,분양 세대수,...,6~10층 동수,6~10층 세대수,11~15층 동수,11~15층 세대수,16~20층 동수,16~20층 세대수,21층이상 동수,21층이상 세대수,~40㎡ 세대수,40㎡~60㎡ 세대수,60㎡~85㎡ 세대수,85㎡~102㎡ 세대수,102㎡~135㎡ 세대수,135㎡~ 세대수,서울특별시_부동산소비심리지수,미분양_구,시점,기준금리,국고채,회사채,국고채-회사채,생산자물가지수,소비자물가지수,서울특별시_가계대출,서울특별시_주택담보대출,서울특별시_주택매매거래량,서울특별시_아파트매매거래량,전국매매가격지수,전국전세가격지수,서울특별시_주택인허가실적,서울특별시_주택착공실적,서울특별시_주택준공실적,서울특별시_주택미분양,미분양_서울특별시,지번주소,공시가격기반단지명,동수_y,세대수,사용승인일자,소형학원수,중형학원수,대형학원수,종합학원수,입시학원수,외국어학원수,예체능학원수,독서실수,정보학원수,특수교육학원수,직업기술학원수,기타학원수,번지_jm,좌표X_jm,좌표Y_jm,도로명_nan,도로명주소_nan,번지주소_2,도로명주소_추가,도로명_df2,도로명_df
0,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.97,201712,8,3,1987,언주로 3,124000.0,0,서울특별시 강남구 개포동 658-1,서울특별시 강남구 개포동 658-1 3층,2017-12-08,강남구,개포동,서울특별시 강남구 언주로 3,2017,40291.0,A10023565,개포6차우성아파트,아파트,서울특별시 강남구 언주로 3,강남구,개포동,언주로,8.0,270.0,우성건설,1987-11-21,22637.0,20204.0,22637.0,20.0,250.0,0.0,NaN,262.0,127.05721,37.476763,127.058091,37.476367,서울특별시 강남구 개포동 658-1,127.05721,37.476763,서울특별시 강남구 언주로 3,127.05721,37.476763,0.061744,개포우성아파트,1.129065,구룡,분양,6차우성,서울특별시 강남구 개포동 658-1,8.0,270.0,270.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0,250.0,NaN,NaN,NaN,94.2,0.0,201712,1.5,2.1,2.649,0.549,99.09,97.698,282438.7,176991.0,13740.0,8484.0,87.701,93.042,113131.0,13157.0,5968.0,45.0,45.0,서울특별시 강남구 개포동 658-1,개포6차우성아파트1동~8동,8.0,270.0,19871121.0,39.0,3.0,0.0,2.0,23.0,1.0,14.0,0.0,0.0,0.0,1.0,1.0,658-1,127.056859,37.476276,언주로 3,서울특별시 강남구 언주로 3,서울특별시 강남구 개포동 658-1,NaN,서울특별시 강남구 언주로 3,서울특별시 강남구 언주로 3
1,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.97,201712,22,4,1987,언주로 3,123500.0,0,서울특별시 강남구 개포동 658-1,서울특별시 강남구 개포동 658-1 4층,2017-12-22,강남구,개포동,서울특별시 강남구 언주로 3,2017,40291.0,A10023565,개포6차우성아파트,아파트,서울특별시 강남구 언주로 3,강남구,개포동,언주로,8.0,270.0,우성건설,1987-11-21,22637.0,20204.0,22637.0,20.0,250.0,0.0,NaN,262.0,127.05721,37.476763,127.058091,37.476367,서울특별시 강남구 개포동 658-1,127.05721,37.476763,서울특별시 강남구 언주로 3,127.05721,37.476763,0.061744,개포우성아파트,1.129065,구룡,분양,6차우성,서울특별시 강남구 개포동 658-1,8.0,270.0,270.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0,250.0,NaN,NaN,NaN,94.2,0.0,201712,1.5,2.1,2.649,0.549,99.09,97.698,282438.7,176991.0,13740.0,8484.0,87.701,93.042,113131.0,13157.0,5968.0,45.0,45.0,서울특별시 강남구 개포동 658-1,개포6차우성아파트1동~8동,8.0,270.0,19871121.0,39.0,3.0,0.0,2.0,23.0,1.0,14.0,0.0,0.0,0.0,1.0,1.0,658-1,127.056859,37.476276,언주로 3,서울특별시 강남구 언주로 3,서울특별시 강남구 개포동 658-1,NaN,서울특별시 강남구 언주로 3,서울특별시 강남구 언주로 3


In [513]:
df2[['도로명_df2', '도로명_df']]

,도로명_df2,도로명_df
0,서울특별시 강남구 언주로 3,서울특별시 강남구 언주로 3
1,서울특별시 강남구 언주로 3,서울특별시 강남구 언주로 3
2,서울특별시 강남구 언주로 3,서울특별시 강남구 언주로 3
3,서울특별시 강남구 언주로 3,서울특별시 강남구 언주로 3
4,서울특별시 강남구 언주로 3,서울특별시 강남구 언주로 3
...,...,...
1128089,서울특별시 마포구 토정로17길 6,서울특별시 중랑구 신내역로1길 85
1128090,서울특별시 성동구 송정길 32,서울특별시 중랑구 신내역로1길 85
1128091,서울특별시 중구 마른내로 79,서울특별시 중랑구 신내역로1길 85
1128092,서울특별시 중구 마른내로 79,서울특별시 중랑구 신내역로1길 85


In [541]:
df = pd.read_csv('../data/trainset_240716.csv')

In [542]:
# 1번열과 2번열의 문장을 결합하여 3번열 생성
df['번지주소_2'] = df['시군구'] + ' ' + df['번지_jm']

# 결과 출력
df.head(2)

,시군구,번지,본번,부번,아파트명,전용면적(㎡),계약년월,계약일,층,건축년도,도로명,target,is_test,번지주소,id,계약년월일,구,동,도로명주소,계약년,번호,k-아파트코드,k-아파트명,"k-단지분류(아파트,주상복합등등)",kapt도로명주소,주소(시군구),주소(읍면동),주소(도로명),k-전체동수,k-전체세대수,k-건설사(시공사),준공일자,k-연면적,k-주거전용면적,k-관리비부과면적,k-전용면적별세대현황(60㎡이하),k-전용면적별세대현황(60㎡~85㎡이하),k-85㎡~135㎡이하,k-135㎡초과,주차대수,좌표X,좌표Y,출입구_x좌표_wgs84,출입구_y좌표_wgs84,지번주소_x,final_X,final_Y,Property_ID,X,Y,Nearest_Bus_Station_Distance,Nearest_Bus_Station_Name,Nearest_Subway_Station_Distance,Nearest_Subway_Station_Name,입주유형,단지명,지번주소_y,동수_x,합계 세대수,분양 세대수,...,합계 동수,5층 동수,5층 세대수,6~10층 동수,6~10층 세대수,11~15층 동수,11~15층 세대수,16~20층 동수,16~20층 세대수,21층이상 동수,21층이상 세대수,~40㎡ 세대수,40㎡~60㎡ 세대수,60㎡~85㎡ 세대수,85㎡~102㎡ 세대수,102㎡~135㎡ 세대수,135㎡~ 세대수,서울특별시_부동산소비심리지수,미분양_구,시점,기준금리,국고채,회사채,국고채-회사채,생산자물가지수,소비자물가지수,서울특별시_가계대출,서울특별시_주택담보대출,서울특별시_주택매매거래량,서울특별시_아파트매매거래량,전국매매가격지수,전국전세가격지수,서울특별시_주택인허가실적,서울특별시_주택착공실적,서울특별시_주택준공실적,서울특별시_주택미분양,미분양_서울특별시,지번주소,공시가격기반단지명,동수_y,세대수,사용승인일자,소형학원수,중형학원수,대형학원수,종합학원수,입시학원수,외국어학원수,예체능학원수,독서실수,정보학원수,특수교육학원수,직업기술학원수,기타학원수,번지_jm,좌표X_jm,좌표Y_jm,도로명_nan,도로명주소_nan,번지주소_2
0,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.97,201712,8,3,1987,언주로 3,124000.0,0,서울특별시 강남구 개포동 658-1,서울특별시 강남구 개포동 658-1 3층,2017-12-08,강남구,개포동,서울특별시 강남구 언주로 3,2017,40291.0,A10023565,개포6차우성아파트,아파트,서울특별시 강남구 언주로 3,강남구,개포동,언주로,8.0,270.0,우성건설,1987-11-21,22637.0,20204.0,22637.0,20.0,250.0,0.0,NaN,262.0,127.05721,37.476763,127.058091,37.476367,서울특별시 강남구 개포동 658-1,127.05721,37.476763,서울특별시 강남구 언주로 3,127.05721,37.476763,0.061744,개포우성아파트,1.129065,구룡,분양,6차우성,서울특별시 강남구 개포동 658-1,8.0,270.0,270.0,...,8.0,8.0,270.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0,250.0,NaN,NaN,NaN,94.2,0.0,201712,1.5,2.1,2.649,0.549,99.09,97.698,282438.7,176991.0,13740.0,8484.0,87.701,93.042,113131.0,13157.0,5968.0,45.0,45.0,서울특별시 강남구 개포동 658-1,개포6차우성아파트1동~8동,8.0,270.0,19871121.0,39.0,3.0,0.0,2.0,23.0,1.0,14.0,0.0,0.0,0.0,1.0,1.0,658-1,127.056859,37.476276,언주로 3,서울특별시 강남구 언주로 3,서울특별시 강남구 개포동 658-1
1,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.97,201712,22,4,1987,언주로 3,123500.0,0,서울특별시 강남구 개포동 658-1,서울특별시 강남구 개포동 658-1 4층,2017-12-22,강남구,개포동,서울특별시 강남구 언주로 3,2017,40291.0,A10023565,개포6차우성아파트,아파트,서울특별시 강남구 언주로 3,강남구,개포동,언주로,8.0,270.0,우성건설,1987-11-21,22637.0,20204.0,22637.0,20.0,250.0,0.0,NaN,262.0,127.05721,37.476763,127.058091,37.476367,서울특별시 강남구 개포동 658-1,127.05721,37.476763,서울특별시 강남구 언주로 3,127.05721,37.476763,0.061744,개포우성아파트,1.129065,구룡,분양,6차우성,서울특별시 강남구 개포동 658-1,8.0,270.0,270.0,...,8.0,8.0,270.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0,250.0,NaN,NaN,NaN,94.2,0.0,201712,1.5,2.1,2.649,0.549,99.09,97.698,282438.7,176991.0,13740.0,8484.0,87.701,93.042,113131.0,13157.0,5968.0,45.0,45.0,서울특별시 강남구 개포동 658-1,개포6차우성아파트1동~8동,8.0,270.0,19871121.0,39.0,3.0,0.0,2.0,23.0,1.0,14.0,0.0,0.0,0.0,1.0,1.0,658-1,127.056859,37.476276,언주로 3,서울특별시 강남구 언주로 3,서울특별시 강남구 개포동 658-1


In [543]:
df['번지주소_2'].isna().sum()

0

In [544]:
df_col = df.columns

In [545]:
df_col

Index(['시군구', '번지', '본번', '부번', '아파트명', '전용면적(㎡)', '계약년월', '계약일', '층', '건축년도',
       ...
       '정보학원수', '특수교육학원수', '직업기술학원수', '기타학원수', '번지_jm', '좌표X_jm', '좌표Y_jm',
       '도로명_nan', '도로명주소_nan', '번지주소_2'],
      dtype='object', length=122)

In [547]:
df2 = df.drop(columns=['시군구', '번지', '본번', '부번', '번지주소', '구', '동', 'kapt도로명주소', '주소(시군구)', '주소(읍면동)', '주소(도로명)', '지번주소_y','지번주소','도로명_nan'])

In [578]:
df2[12:15]

,아파트명,전용면적(㎡),계약년월,계약일,층,건축년도,도로명,target,is_test,id,계약년월일,도로명주소,계약년,번호,k-아파트코드,k-아파트명,"k-단지분류(아파트,주상복합등등)",k-전체동수,k-전체세대수,k-건설사(시공사),준공일자,k-연면적,k-주거전용면적,k-관리비부과면적,k-전용면적별세대현황(60㎡이하),k-전용면적별세대현황(60㎡~85㎡이하),k-85㎡~135㎡이하,k-135㎡초과,주차대수,좌표X,좌표Y,출입구_x좌표_wgs84,출입구_y좌표_wgs84,지번주소_x,final_X,final_Y,Property_ID,X,Y,Nearest_Bus_Station_Distance,Nearest_Bus_Station_Name,Nearest_Subway_Station_Distance,Nearest_Subway_Station_Name,입주유형,단지명,동수_x,합계 세대수,분양 세대수,임대 세대수,토지 면적(㎡),합계 동수,5층 동수,5층 세대수,6~10층 동수,6~10층 세대수,11~15층 동수,11~15층 세대수,16~20층 동수,16~20층 세대수,21층이상 동수,21층이상 세대수,~40㎡ 세대수,40㎡~60㎡ 세대수,60㎡~85㎡ 세대수,85㎡~102㎡ 세대수,102㎡~135㎡ 세대수,135㎡~ 세대수,서울특별시_부동산소비심리지수,미분양_구,시점,기준금리,국고채,회사채,국고채-회사채,생산자물가지수,소비자물가지수,서울특별시_가계대출,서울특별시_주택담보대출,서울특별시_주택매매거래량,서울특별시_아파트매매거래량,전국매매가격지수,전국전세가격지수,서울특별시_주택인허가실적,서울특별시_주택착공실적,서울특별시_주택준공실적,서울특별시_주택미분양,미분양_서울특별시,공시가격기반단지명,동수_y,세대수,사용승인일자,소형학원수,중형학원수,대형학원수,종합학원수,입시학원수,외국어학원수,예체능학원수,독서실수,정보학원수,특수교육학원수,직업기술학원수,기타학원수,번지_jm,좌표X_jm,좌표Y_jm,도로명주소_nan,번지주소_2
12,개포우성3차,133.46,201710,13,1,1984,개포로 307,152000.0,0,서울특별시 강남구 개포동 652 1층,2017-10-13,서울특별시 강남구 개포로 307,2017,882.0,A13524004,개포우성3차,아파트,5.0,405.0,우성건설,1984-12-31,63304.0,53437.0,61064.0,0.0,0.0,285.0,NaN,419.0,127.05599,37.483894,127.05694,37.483613,서울특별시 강남구 개포동 652,127.05599,37.483894,서울특별시 강남구 개포로 307,127.05599,37.483894,0.098221,우성3차아파트,0.413486,구룡,분양,개포우성3차,5.0,405.0,405.0,NaN,61064.0,5.0,NaN,NaN,NaN,NaN,5.0,405.0,NaN,NaN,NaN,NaN,NaN,NaN,135.0,NaN,150.0,120.0,105.6,0.0,201710,1.25,2.026,2.574,0.548,99.14,98.077,278568.7,175427.9,8561.0,3942.0,87.569,93.070,89283.0,4269.0,3299.0,56.0,56.0,개포우성3차,5.0,405.0,19841231.0,66.0,14.0,1.0,2.0,42.0,1.0,28.0,3.0,0.0,0.0,0.0,5.0,652,127.056019,37.483975,서울특별시 강남구 개포로 307,서울특별시 강남구 개포동 652
13,개포우성3차,161.00,201711,11,8,1984,개포로 307,182000.0,0,서울특별시 강남구 개포동 652 8층,2017-11-11,서울특별시 강남구 개포로 307,2017,882.0,A13524004,개포우성3차,아파트,5.0,405.0,우성건설,1984-12-31,63304.0,53437.0,61064.0,0.0,0.0,285.0,NaN,419.0,127.05599,37.483894,127.05694,37.483613,서울특별시 강남구 개포동 652,127.05599,37.483894,서울특별시 강남구 개포로 307,127.05599,37.483894,0.098221,우성3차아파트,0.413486,구룡,분양,개포우성3차,5.0,405.0,405.0,NaN,61064.0,5.0,NaN,NaN,NaN,NaN,5.0,405.0,NaN,NaN,NaN,NaN,NaN,NaN,135.0,NaN,150.0,120.0,101.2,0.0,201711,1.50,2.150,2.709,0.559,99.04,97.347,281452.2,176351.5,12041.0,6679.0,87.627,93.103,97080.0,9674.0,5732.0,68.0,68.0,개포우성3차,5.0,405.0,19841231.0,66.0,14.0,1.0,2.0,42.0,1.0,28.0,3.0,0.0,0.0,0.0,5.0,652,127.056019,37.483975,서울특별시 강남구 개포로 307,서울특별시 강남구 개포동 652
14,개포우성3차,161.00,201711,27,8,1984,개포로 307,170000.0,0,서울특별시 강남구 개포동 652 8층,2017-11-27,서울특별시 강남구 개포로 307,2017,882.0,A13524004,개포우성3차,아파트,5.0,405.0,우성건설,1984-12-31,63304.0,53437.0,61064.0,0.0,0.0,285.0,NaN,419.0,127.05599,37.483894,127.05694,37.483613,서울특별시 강남구 개포동 652,127.05599,37.483894,서울특별시 강남구 개포로 307,127.05599,37.483894,0.098221,우성3차아파트,0.413486,구룡,분양,개포우성3차,5.0,405.0,405.0,NaN,61064.0,5.0,NaN,NaN,NaN,NaN,5.0,405.0,NaN,NaN,NaN,NaN,NaN,NaN,135.0,NaN,150.0,120.0,101.2,0.0,201711,1.50,2.150,2.709,0.559,99.04,97.347,281452.2,176351.5,12041.0,6679.0,87.627,93.103,97080.0,9674.0,5732.0,68.0,68.0,개포우성3차,5.0,405.0,19841231.0,66.0,14.0,1.0,2.0,42.0,1.0,28.0,3.0,0.0,0.0,0.0,5.0,652,127.056019,37.483975,서울특별시 강남구 개포로 307,서울특별시 강남구 개포동 652


In [549]:
df = df2.copy()

In [554]:
df[(df['아파트명'].isna()) & (df['단지명'].isna())][['아파트명', '단지명']].isna().sum()

아파트명    2094
단지명     2094
dtype: int64

In [555]:
df['단지명'].isna().sum()

298650

In [557]:
df['아파트명'].isna().sum()

2136

In [574]:
new_df = df[['번지주소_2']].copy()

In [575]:
new_df

,번지주소_2
0,서울특별시 강남구 개포동 658-1
1,서울특별시 강남구 개포동 658-1
2,서울특별시 강남구 개포동 658-1
3,서울특별시 강남구 개포동 658-1
4,서울특별시 강남구 개포동 658-1
...,...
1128089,서울특별시 중랑구 신내동 816
1128090,서울특별시 중랑구 신내동 816
1128091,서울특별시 중랑구 신내동 816
1128092,서울특별시 중랑구 신내동 816


In [585]:
result_df_2

,번지주소_2,도로명주소_추가
0,서울특별시 성동구 하왕십리동 1071,서울특별시 성동구 난계로 100
1,서울특별시 성북구 보문동4가 116,서울특별시 성북구 고려대로2길 37
2,서울특별시 영등포구 당산동4가 99,서울특별시 영등포구 영신로 237
3,서울특별시 용산구 한강로1가 231-23,서울특별시 용산구 한강대로 174-2
4,서울특별시 중구 만리동2가 288,서울특별시 중구 만리재로 177
...,...,...
65,서울특별시 서초구 서초동 1312-3,서울특별시 서초구 서운로 197
66,서울특별시 성동구 옥수동 436,서울특별시 성동구 독서당로 173
67,서울특별시 성동구 옥수동 100,서울특별시 성동구 한림말길 48
68,서울특별시 양천구 목동 925,서울특별시 양천구 목동로 186


In [583]:
new_df = df[['번지주소_2']].copy()

In [584]:
new_df

,번지주소_2
0,서울특별시 강남구 개포동 658-1
1,서울특별시 강남구 개포동 658-1
2,서울특별시 강남구 개포동 658-1
3,서울특별시 강남구 개포동 658-1
4,서울특별시 강남구 개포동 658-1
...,...
1128089,서울특별시 중랑구 신내동 816
1128090,서울특별시 중랑구 신내동 816
1128091,서울특별시 중랑구 신내동 816
1128092,서울특별시 중랑구 신내동 816


In [587]:
# df1의 '도로명'을 기준으로 df2에서 일치하는 값을 찾아 '도로명_추가' 열에 추가
new_df['도로명주소_추가'] = new_df['번지주소_2'].apply(lambda x: result_df_2.loc[result_df_2['번지주소_2'] == x, '도로명주소_추가'].values[0] if x in result_df_2['번지주소_2'].values else np.nan)
new_df

,번지주소_2,도로명주소_추가
0,서울특별시 강남구 개포동 658-1,NaN
1,서울특별시 강남구 개포동 658-1,NaN
2,서울특별시 강남구 개포동 658-1,NaN
3,서울특별시 강남구 개포동 658-1,NaN
4,서울특별시 강남구 개포동 658-1,NaN
...,...,...
1128089,서울특별시 중랑구 신내동 816,서울특별시 중랑구 신내역로1길 85
1128090,서울특별시 중랑구 신내동 816,서울특별시 중랑구 신내역로1길 85
1128091,서울특별시 중랑구 신내동 816,서울특별시 중랑구 신내역로1길 85
1128092,서울특별시 중랑구 신내동 816,서울특별시 중랑구 신내역로1길 85


In [594]:
new_df['도로명주소_추가'].notna().sum()

19812

In [590]:
new_df['df'] = df['번지주소_2'].copy()

In [593]:
new_df[:50]

,번지주소_2,도로명주소_추가,df
0,서울특별시 강남구 개포동 658-1,NaN,서울특별시 강남구 개포동 658-1
1,서울특별시 강남구 개포동 658-1,NaN,서울특별시 강남구 개포동 658-1
2,서울특별시 강남구 개포동 658-1,NaN,서울특별시 강남구 개포동 658-1
3,서울특별시 강남구 개포동 658-1,NaN,서울특별시 강남구 개포동 658-1
4,서울특별시 강남구 개포동 658-1,NaN,서울특별시 강남구 개포동 658-1
5,서울특별시 강남구 개포동 658-1,NaN,서울특별시 강남구 개포동 658-1
6,서울특별시 강남구 개포동 658-1,NaN,서울특별시 강남구 개포동 658-1
7,서울특별시 강남구 개포동 658-1,NaN,서울특별시 강남구 개포동 658-1
8,서울특별시 강남구 개포동 658-1,NaN,서울특별시 강남구 개포동 658-1
9,서울특별시 강남구 개포동 658-1,NaN,서울특별시 강남구 개포동 658-1


In [592]:
# 두 열의 값이 다른 행의 인덱스 추출
different_rows = new_df[new_df['번지주소_2'] != new_df['df']].index

print("값이 다른 행의 인덱스:", different_rows)

값이 다른 행의 인덱스: Index([], dtype='int64')


In [611]:
new_df['도로명주소_df'] = df['도로명주소_nan'].copy()

In [612]:
test_df = new_df[new_df['도로명주소_추가'].notna()]

In [613]:
# 두 열의 값이 다른 행의 인덱스 추출
different_rows = test_df[test_df['도로명주소_추가'] != test_df['도로명주소_df']].index

print("값이 다른 행의 인덱스:", different_rows)

값이 다른 행의 인덱스: Index([   7311,    7312,    7313,    7314,    7315,    7316,    7317,    7318,
          7319,    7320,
       ...
       1124952, 1126304, 1126305, 1126306, 1126307, 1126308, 1126309, 1126310,
       1127767, 1128004],
      dtype='int64', length=7015)


In [614]:
test_df

,번지주소_2,도로명주소_추가,df,도로명주소_df
1757,서울특별시 강남구 대치동 66,서울특별시 강남구 영동대로 210,서울특별시 강남구 대치동 66,서울특별시 강남구 영동대로 210
1758,서울특별시 강남구 대치동 66,서울특별시 강남구 영동대로 210,서울특별시 강남구 대치동 66,서울특별시 강남구 영동대로 210
1759,서울특별시 강남구 대치동 66,서울특별시 강남구 영동대로 210,서울특별시 강남구 대치동 66,서울특별시 강남구 영동대로 210
1760,서울특별시 강남구 대치동 66,서울특별시 강남구 영동대로 210,서울특별시 강남구 대치동 66,서울특별시 강남구 영동대로 210
1761,서울특별시 강남구 대치동 66,서울특별시 강남구 영동대로 210,서울특별시 강남구 대치동 66,서울특별시 강남구 영동대로 210
...,...,...,...,...
1128089,서울특별시 중랑구 신내동 816,서울특별시 중랑구 신내역로1길 85,서울특별시 중랑구 신내동 816,서울특별시 중랑구 신내역로1길 85
1128090,서울특별시 중랑구 신내동 816,서울특별시 중랑구 신내역로1길 85,서울특별시 중랑구 신내동 816,서울특별시 중랑구 신내역로1길 85
1128091,서울특별시 중랑구 신내동 816,서울특별시 중랑구 신내역로1길 85,서울특별시 중랑구 신내동 816,서울특별시 중랑구 신내역로1길 85
1128092,서울특별시 중랑구 신내동 816,서울특별시 중랑구 신내역로1길 85,서울특별시 중랑구 신내동 816,서울특별시 중랑구 신내역로1길 85


In [615]:
test_df.loc[different_rows]

,번지주소_2,도로명주소_추가,df,도로명주소_df
7311,서울특별시 강동구 명일동 270,서울특별시 강동구 상암로 214,서울특별시 강동구 명일동 270,서울특별시 강동구 상암로 224
7312,서울특별시 강동구 명일동 270,서울특별시 강동구 상암로 214,서울특별시 강동구 명일동 270,서울특별시 강동구 상암로 224
7313,서울특별시 강동구 명일동 270,서울특별시 강동구 상암로 214,서울특별시 강동구 명일동 270,서울특별시 강동구 상암로 224
7314,서울특별시 강동구 명일동 270,서울특별시 강동구 상암로 214,서울특별시 강동구 명일동 270,서울특별시 강동구 상암로 224
7315,서울특별시 강동구 명일동 270,서울특별시 강동구 상암로 214,서울특별시 강동구 명일동 270,서울특별시 강동구 상암로 225
...,...,...,...,...
1126308,서울특별시 양천구 목동 925,서울특별시 양천구 목동로 186,서울특별시 양천구 목동 925,서울특별시 양천구 목동로 212
1126309,서울특별시 양천구 목동 925,서울특별시 양천구 목동로 186,서울특별시 양천구 목동 925,서울특별시 양천구 목동로 212
1126310,서울특별시 양천구 목동 925,서울특별시 양천구 목동로 186,서울특별시 양천구 목동 925,서울특별시 양천구 목동로 212
1127767,서울특별시 종로구 평창동 596,서울특별시 종로구 평창문화로 170,서울특별시 종로구 평창동 596,서울특별시 종로구 평창문화로 172


In [624]:
new_df['도로명_fill'] = new_df['도로명주소_df'].copy()

In [625]:
new_df['도로명_fill'] = new_df['도로명_fill'].fillna(new_df['도로명주소_추가'])
new_df.loc[different_rows]

,번지주소_2,도로명주소_추가,df,도로명주소_df,도로명_fill
7311,서울특별시 강동구 명일동 270,서울특별시 강동구 상암로 214,서울특별시 강동구 명일동 270,서울특별시 강동구 상암로 224,서울특별시 강동구 상암로 224
7312,서울특별시 강동구 명일동 270,서울특별시 강동구 상암로 214,서울특별시 강동구 명일동 270,서울특별시 강동구 상암로 224,서울특별시 강동구 상암로 224
7313,서울특별시 강동구 명일동 270,서울특별시 강동구 상암로 214,서울특별시 강동구 명일동 270,서울특별시 강동구 상암로 224,서울특별시 강동구 상암로 224
7314,서울특별시 강동구 명일동 270,서울특별시 강동구 상암로 214,서울특별시 강동구 명일동 270,서울특별시 강동구 상암로 224,서울특별시 강동구 상암로 224
7315,서울특별시 강동구 명일동 270,서울특별시 강동구 상암로 214,서울특별시 강동구 명일동 270,서울특별시 강동구 상암로 225,서울특별시 강동구 상암로 225
...,...,...,...,...,...
1126308,서울특별시 양천구 목동 925,서울특별시 양천구 목동로 186,서울특별시 양천구 목동 925,서울특별시 양천구 목동로 212,서울특별시 양천구 목동로 212
1126309,서울특별시 양천구 목동 925,서울특별시 양천구 목동로 186,서울특별시 양천구 목동 925,서울특별시 양천구 목동로 212,서울특별시 양천구 목동로 212
1126310,서울특별시 양천구 목동 925,서울특별시 양천구 목동로 186,서울특별시 양천구 목동 925,서울특별시 양천구 목동로 212,서울특별시 양천구 목동로 212
1127767,서울특별시 종로구 평창동 596,서울특별시 종로구 평창문화로 170,서울특별시 종로구 평창동 596,서울특별시 종로구 평창문화로 172,서울특별시 종로구 평창문화로 172


In [627]:
# 두 열의 값이 다른 행의 인덱스 추출
different_rows = new_df[new_df['도로명주소_df'] != new_df['도로명_fill']].index

print("값이 다른 행의 인덱스:", different_rows)

값이 다른 행의 인덱스: Index([  63981,   63986,   63987,   67838,   67842,   67845,   67846,   67847,
         67850,   67851,
       ...
       1115469, 1115470, 1115471, 1115472, 1115595, 1115601, 1115602, 1123728,
       1124312, 1124313],
      dtype='int64', length=2339)


In [635]:
new_df.loc[different_rows, '도로명_fill']

63981          서울특별시 성동구 난계로 100
63986          서울특별시 성동구 난계로 100
63987          서울특별시 성동구 난계로 100
67838        서울특별시 성북구 고려대로2길 37
67842        서울특별시 성북구 고려대로2길 37
                   ...          
1115601      서울특별시 종로구 평창문화로 170
1115602      서울특별시 종로구 평창문화로 170
1123728    서울특별시 서대문구 이화여대8길 123
1124312       서울특별시 서초구 반포대로 333
1124313       서울특별시 서초구 반포대로 333
Name: 도로명_fill, Length: 2339, dtype: object

In [640]:
test_df[test_df['도로명주소_df'].isna()]

,번지주소_2,도로명주소_추가,df,도로명주소_df,도로명_fill
63981,서울특별시 성동구 하왕십리동 1071,서울특별시 성동구 난계로 100,서울특별시 성동구 하왕십리동 1071,NaN,서울특별시 성동구 난계로 100
63986,서울특별시 성동구 하왕십리동 1071,서울특별시 성동구 난계로 100,서울특별시 성동구 하왕십리동 1071,NaN,서울특별시 성동구 난계로 100
63987,서울특별시 성동구 하왕십리동 1071,서울특별시 성동구 난계로 100,서울특별시 성동구 하왕십리동 1071,NaN,서울특별시 성동구 난계로 100
67838,서울특별시 성북구 보문동4가 116,서울특별시 성북구 고려대로2길 37,서울특별시 성북구 보문동4가 116,NaN,서울특별시 성북구 고려대로2길 37
67842,서울특별시 성북구 보문동4가 116,서울특별시 성북구 고려대로2길 37,서울특별시 성북구 보문동4가 116,NaN,서울특별시 성북구 고려대로2길 37
...,...,...,...,...,...
1115601,서울특별시 종로구 평창동 596,서울특별시 종로구 평창문화로 170,서울특별시 종로구 평창동 596,NaN,서울특별시 종로구 평창문화로 170
1115602,서울특별시 종로구 평창동 596,서울특별시 종로구 평창문화로 170,서울특별시 종로구 평창동 596,NaN,서울특별시 종로구 평창문화로 170
1123728,서울특별시 서대문구 북아현동 1017,서울특별시 서대문구 이화여대8길 123,서울특별시 서대문구 북아현동 1017,NaN,서울특별시 서대문구 이화여대8길 123
1124312,서울특별시 서초구 반포동 1-1,서울특별시 서초구 반포대로 333,서울특별시 서초구 반포동 1-1,NaN,서울특별시 서초구 반포대로 333


In [643]:
df['도로명주소_nan'].isna().sum()

2339

In [646]:
new_df['도로명주소_df'].isna().sum()

2339

In [647]:
new_df['도로명_fill'].isna().sum()

991

In [656]:
new_df_2 = new_df[['번지주소_2', '도로명_fill']]
new_df_2.to_csv('../data/addr_up_ej.csv', index=False)

In [653]:
new_df_2

,번지주소_2,도로명_fill
0,서울특별시 강남구 개포동 658-1,서울특별시 강남구 언주로 3
1,서울특별시 강남구 개포동 658-1,서울특별시 강남구 언주로 3
2,서울특별시 강남구 개포동 658-1,서울특별시 강남구 언주로 3
3,서울특별시 강남구 개포동 658-1,서울특별시 강남구 언주로 3
4,서울특별시 강남구 개포동 658-1,서울특별시 강남구 언주로 3
...,...,...
1128089,서울특별시 중랑구 신내동 816,서울특별시 중랑구 신내역로1길 85
1128090,서울특별시 중랑구 신내동 816,서울특별시 중랑구 신내역로1길 85
1128091,서울특별시 중랑구 신내동 816,서울특별시 중랑구 신내역로1길 85
1128092,서울특별시 중랑구 신내동 816,서울특별시 중랑구 신내역로1길 85


In [658]:
df.to_csv('../data/train_240716_gogo.csv', index=False)